# **Recon Algorithm**

This code comes from https://github.com/ArnaoutLab/Recon. It is only slightly adapted to use in a jupyter notebook format here.

# Imports & Google Drive Setup

In [ ]:
import argparse # added
from argparse import ArgumentParser, REMAINDER, RawDescriptionHelpFormatter
from ast import literal_eval
from collections import defaultdict
# from subprocess import getoutput
import subprocess
from datetime import datetime
from math import isnan, log, ceil, floor, exp, factorial
from numpy import sqrt, array, interp, mean, arange
from os.path import expanduser, isfile, isdir, dirname, abspath, exists, isabs
from os import listdir, system, getcwd
# from scipy.optimize.lbfgsb import _minimize_lbfgsb as minimize # Deprecated, changed to line below
from scipy.optimize import minimize
from sys import argv, exit, stdout
from time import time
from traceback import print_exc
import warnings

#new
from collections import Counter

# added
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from google.colab import drive
from pathlib import Path
import os

In [ ]:
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
PATH = "/content/drive/MyDrive/Data Science/Thesis/Recon"
os.chdir(PATH)

In [ ]:
pwd

'/content/drive/MyDrive/Data Science/Thesis/Recon'

# Code to run (Adapted from Parser)

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
verbose = True # print verbose output to stdout
very_verbose = True # print even more verbose output to stdout
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
file_in = ['df_fill_in_1_5.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


# Parser

In [ ]:
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter) # added

# formatter_class=RawDescriptionHelpFormatter) # try to maintain alphabetical order a-zA-z # removed
parser.add_argument('-a', '--aicc_multiple', type=float, default=1.3, help='multiple of the observed number of datapoints used for AICc')
parser.add_argument('-b', '--precomputed_error_bar_file', type=str, help='file name for precomputed error bars')
parser.add_argument('-c', '--clone_distribution_in_file', action='store_true', help='input file is of the form clone_size tab number_of_clones of that size')
parser.add_argument('-d', '--bin_size', type=int, default=1, help='average number of observations for each individual')
parser.add_argument('-df', '--D_number_file', default=None, type=str, help='D-number file (output of -D option)')
parser.add_argument('-e', '--make_error_bars', action='store_true', help='make a precomputed error bar file that can be used to calculate error bars')
parser.add_argument('-f', '--fraction_small_clones', default=0.1, type=float, help='see get_sampling_limit')
parser.add_argument('-ff', '--fit_file', default=None, type=str, help='Recon fit-file (output of -R option)')
parser.add_argument("-j", "--javascript_file", default="plot_clone_size_distribution_ref.js", help="reference javascript file, for plotting resample")
parser.add_argument('-l', '--parameter_limit', default=20, type=int, help='maximum number of parameters to fit')
parser.add_argument('-m', '--min_number_of_doublets', default=100, type=int, help='see get_sampling_limit')
parser.add_argument('-n', '--number_of_error_bars', default=2., type=float, help='mapping between gold-standard error bar fits and standard deviations in power calculation')
parser.add_argument('-o', '--file_out', default="test_recon_output.txt", type=str, help='output file for fit')
parser.add_argument('-p', '--make_power_table', action='store_true', help='output a table of cells required to see fold difference')
parser.add_argument('-q', '--q', type=str, default=0., help='Hill number parameter to use when making table for power calculations')
parser.add_argument('-r', '--resample', action='store_true', help='output a resampling of the model fit with the given fit file')
parser.add_argument('-s', '--noise_test_ratio_threshold', default=3., type=float, help='for avoiding fitting noise')
parser.add_argument('-ss', '--sample_size', default=None, type=float, help='sample size, for calculating U_max')
parser.add_argument('-t', '--threshold', type=int, default=30, help='largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed')
parser.add_argument('-u', '--cutoff_score', type=float, default=0.1, help='for stopping iteration when estimated clone distribution is at a fixed point')
parser.add_argument('-v', '--verbose', action='store_true', help='print verbose output to stdout')
parser.add_argument('-vv', '--very_verbose', action='store_true', help='print even more verbose output to stdout')
parser.add_argument('-x', '--make_resample_plot', action='store_true', help="plots resample of fit against original sample")
parser.add_argument('-A', '--total_clones_in_overall_population', type=float, default=None, help='total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution (optionally)')
parser.add_argument('-C', '--list_of_number_of_clones', nargs='+', default=[1e4, 3e4, 1e5, 1e6, 3e6], help='column headers for power table')
parser.add_argument('-D', '--make_table_of_D_numbers', action='store_true', help='output a table of reconstructed D numbers with error bars from input fit files.')
parser.add_argument('-F', '--list_of_fold_differences', nargs='+', default=[1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.], help='row titles for power table')
parser.add_argument('-N', '--N', default=None, type=float, help='overall population size, for calculating U_max')
parser.add_argument('-Q', '--qs', nargs='+', default=[0., 1., 2., float('inf')], help='space-separated list of Hill-number parameters for making error-bar files and table of D numbers')
parser.add_argument('-R', '--run_recon', action='store_true', help='run Recon on sample to generate overall distribution parameters and fit file')
parser.add_argument('-S', '--initial_scale_factors', nargs='+', default=[0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1], help='for determining means')
parser.add_argument('-U', '--resource_path', type=str, default = expanduser('~')+"/Desktop", help='folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js')
parser.add_argument('-Umax', '--upper_bound', action='store_true', help='calculate U_max')
parser.add_argument('-W', '--test_weights_list', nargs='+', default=[0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6], help='list of weights to attempt to add')
parser.add_argument('-X', '--x_max', type=float, default=None, help='the x_max for the output plot')
parser.add_argument('-Y', '--y_max', type=float, default=None, help='the y_max for the output plot (note, log scale)')

parser.add_argument('file_in', nargs=REMAINDER, help='list of file names containing clone data. A long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files')

args = parser.parse_args()
globals().update(vars(args))
if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


# Classes

In [ ]:
# CLASSES ===============================================

class mixed_distribution:
    """
    A mixed_distribution object represents a mixed Poisson
    distribution.
    
    A mixed_distribution is defined by a sum of Poisson distributions
    where the weight and mean of each subdistribution must be specified
    and the weights must sum to one.
    """
    def __init__(self,weight_mean_pairs):
        total_weight = float(sum(k[0] for k in weight_mean_pairs)) # Normalise weights to 1.
        self.weights = [k[0]/total_weight for k in weight_mean_pairs]
        self.means = [float(k[1]) for k in weight_mean_pairs]
        self.weight_mean_pairs = list(zip(self.weights, self.means))

    def value_at_clone_size(self,i):
        vals = [weight*poisson_pmf(mean, i) for weight, mean in self.weight_mean_pairs]
        return sum(vals)


class sample_from_parent:
    def __init__(self, weights, means, sample_size, total_clones, population_size, total_observed_clones, estimated_n0, fitted_weights, fitted_means):
        try:
            total_weight = float(sum(weights)) # Normalise weights to 1.0
            self.weights = [weight/total_weight for weight in weights]
        except: self.weights = weights
        self.means = means
        self.sample_size = sample_size
        self.total_clones = total_clones
        self.population_size = population_size
        self.fitted_weights = fitted_weights
        self.fitted_means = fitted_means
        self.total_observed_clones = total_observed_clones
        self.estimated_n0 = estimated_n0

# Functions

In [ ]:
# 1. FUNCTIONS =====================================

max_clone_size_for_which_factorials_can_be_converted_to_float = 170 # empirical platform limit
factorials = [factorial(k) for k in range( max_clone_size_for_which_factorials_can_be_converted_to_float+1 )] # need to go beyond threshold for when sample_from_model uses it
epsilon = 0.001
def poisson_pmf(mu, k): 
    try: return exp(-mu) * mu**k / factorials[k]
    except IndexError: return epsilon # if the factorial is too big, assume the result will be zero. Return an epsilon so eventually the clone limit will be reached; otherwise risk an infinite loop
    except OverflowError: pass
    try: return exp( -mu + k*log(mu) - log(factorials[k]) ) # triggered if OverflowError (number too large). Get around that by taking exp(log())
    except: pass
    return epsilon # to prevent an infinite loop 



## 1.1 Functions for Reconstruction

In [ ]:
# 1.1 functions for reconstruction ==================

def calculate_AICc(AIC, number_of_parameters, effective_number_of_data_points):
    """
    AIC is the Aikaike Information Criterion and AICc the corrected
    criterion for small numbers of observations
    """
    return AIC + 2.*number_of_parameters*(number_of_parameters + 1.) / (effective_number_of_data_points - number_of_parameters - 1.)


def read_observed_clone_size_distribution_v2(file_in, clone_distribution_in_file, threshold):
    """
    Return dict[clones size] = count
    """
    # with open(file_in,'rU') as f:
    with open(file_in) as f:
        observed_clone_size_distribution = defaultdict(int)
        if clone_distribution_in_file: # Format from literature
            for line in f:
            	# below was originally written for python 2
                # if line.startswith("#") or strip(line) == "": continue
                # if line.startswith("#") or line.strip() == "": continue
                if line.startswith("#") or line.strip() == "": continue # changed because my data lines are already strings -Carla
                # if line.startswith("#") or line.str.strip() == "": continue
                try:
                    # below was originally written for python 2
                    # clone_size, no = split(strip(line), "\t")
                    clone_size, no = (line.strip()).split("\t")
                except ValueError:
                    print("Error: this line in the input file does not contain two columns separated by a tab:\n")
                    print(line)
                    print("\nExiting...")
                    exit()
                # try-except the ints
                if clone_size.isdigit() and no.isdigit():
                	clone_size = int(clone_size)
                	no = int(no)
                else:
                    print("Error: either the clone_size '%s' or the no '%s' is not an integer. These must be integers. Exiting..." % (clone_size, no))
                    exit()
                observed_clone_size_distribution[clone_size] = int(no)
        else:
            species_count_hash = defaultdict(int)
            for line in f:
            	# below was originally written for python 2
            	# if line.startswith("#") or strip(line) == "": continue
                if line.startswith("#") or line.strip() == "": continue
                try:
                    # below was originally written for python 2
                    # species, no = split(strip(line), "\t")
                    species, no = (line.strip()).split("\t")
                except ValueError:
                    print("Error: this line in the input file does not contain two columns separated by a tab:\n")
                    print(line)
                    print("\nExiting...")
                    exit()
                if no.isdigit():
                    no = int(no) # check that it's an integer, not just that you can integerize it
                else:
                    print("Error: the count for species '%s' is '%s', which appears to be not an integer. Counts must be integers. Exiting..." % (species, no))
                    exit()
                species_count_hash[species] += no
            observed_clone_size_distribution = Counter(list(species_count_hash.values()))
            if verbose: print("species_count_hash:", species_count_hash)
    # if verbose: print "observed_clone_size_distribution:", observed_clone_size_distribution
    # print(observed_clone_size_distribution)
    return observed_clone_size_distribution


def calc_log_likelihood_n0_as_data(estimated_clone_size_distribution, distribution, threshold):
    try:
        log_likelihood = sum( estimated_clone_size_distribution[k]*log(distribution.value_at_clone_size(k)) for k in range(0, threshold) ) # note 0
    except ValueError: # triggered if new_distribution.value_at_clone_size(k) = 0.
        log_likelihood = -float('inf')
        if verbose: print("Log likelihood is -inf!")
    return log_likelihood


def calc_log_likelihood_n0_as_parameter(estimated_clone_size_distribution, distribution, threshold, total_observed_clones):
    try: 
        log_likelihood = sum( estimated_clone_size_distribution[k]*log(distribution.value_at_clone_size(k)) for k in range(1, threshold) ) # note 1
        p_0 = distribution.value_at_clone_size(0)
        log_likelihood -= total_observed_clones * log(1. - p_0)
    except ValueError: # triggered if new_distribution.value_at_clone_size(k) = 0.
        log_likelihood = -float("inf")
        if verbose: print('Log likelihood is -inf!')
    return log_likelihood


def log_likelihood_score(fit_params, estimated_clone_size_distribution):
    # double check
    # weights = list(fit_params[:len(fit_params)/2])
    weights = list(fit_params[:int(len(fit_params)/2)])
    if any(weight > 0. for weight in weights):
        weights = [max(weight, 0.) for weight in weights] # Ensure this is a local variable!
    else:
        weights = [float(weight == max(weights)) for weight in weights]
        if verbose: print('All weights negative', weights)
    weight_norm = sum(weights)
    try:  weights = [weight / weight_norm for weight in weights]
    except ZeroDivisionError: weights = [1.] + weights[1:]
    weights = [1. - sum(weights[1:])] + weights[1:]
    # double check
    # means = list(fit_params[len(fit_params)/2:])
    means = list(fit_params[(int(len(fit_params/2))):]) # Ensure this is a local variable!
    means = [max(mean,1e-6) for mean in means]
    test_distribution = mixed_distribution(list(zip(weights,means)))
    log_likelihood = calc_log_likelihood_n0_as_data(estimated_clone_size_distribution, test_distribution, threshold)
    return -log_likelihood


def MLE_fit_v2(file_out, threshold, new_weights, new_means, observed_clone_size_distribution):
    """
    v2 does not return the initial parameters.
        
    The code which call MLE_fit_v2 already knows these.
    """
    if verbose: 
        print('==== BEGIN FIT ===\ninitial parameters '+str(new_weights+new_means))

    with open(file_out,'a') as f: 
        f.write('initial parameters = %s\n' % (new_weights + new_means))
    total_observed_clones = float(sum([observed_clone_size_distribution[key] for key in observed_clone_size_distribution if key < threshold]))
    new_distribution = mixed_distribution(list(zip(new_weights,new_means)))
    # print(total_observed_clones)
    # print(new_distribution)
    
    total_estimated_clones = round( total_observed_clones/(1.-new_distribution.value_at_clone_size(0)) )
    estimated_n_0 = total_estimated_clones - total_observed_clones
    estimated_clone_size_distribution = defaultdict(int)
    estimated_clone_size_distribution[0] = estimated_n_0

    for key in observed_clone_size_distribution:
        estimated_clone_size_distribution[key] = observed_clone_size_distribution[key]
    count_iterations = 0
    score_function = cutoff_score + 1. # score_function stops the iteration when the estimated clone distribution is at a fixed point.
    dictionary_of_fits = {}
    iteration_limit = 20

    while ( (score_function > cutoff_score) and (count_iterations < iteration_limit) ): # Could implement this as separate cutoffs for weights and means. Begin with one cutoff.
        count_iterations += 1

        # Step 1: Use the estimated_clone_distribution to calculate an
        # updated mixed distribution. This is the MLE of weights and
        # means given the estimated clone distribution. scorefunction2
        # stops the iteration when the weights and means are at a
        # fixed point.

        if verbose:
            print("iteration %i" % count_iterations)
            print("old estimated n0\t%i" % estimated_n_0) # This output gives the estimated clones at the start of the iteration
        old_estimated_n_0 = estimated_n_0
        start_params = array(new_weights+new_means)
        try:
            res = minimize(log_likelihood_score, # "L-BFGS-B"; no bounds or callback
                           start_params, 
                           args=(estimated_clone_size_distribution,))
            # double check
            # new_weights = list(res.x[:len(res.x)/2])
            new_weights = list(res.x[:int(len(res.x)/2)])
            if any(weight > 0 for weight in new_weights): # Useful to record in file?
                new_weights = [max(weight,0.) for weight in new_weights]
            else:
                new_weights = [float(weight == max(new_weights)) for weight in new_weights]
            weight_norm = sum(new_weights)
            new_weights = [weight / weight_norm for weight in new_weights]
            # double check
            # new_means = list(res.x[len(res.x)/2:])
            new_means = list(res.x[int(len(res.x)/2):])
            if any(new_mean < 0. for new_mean in new_means):
                new_means = [max(new_mean, 1e-6) for new_mean in new_means]
        except ValueError:
            if verbose: print("ValueError in minimize!")
            new_weights = start_params[:len(start_params)/2]
            new_means = start_params[len(start_params)/2:]
        if verbose: print("Fitted parameters (new_weights, new_means) %s %s" % (new_weights, new_means))

        # Step 2: Calculate an updated estimated_clone_distribution
        # based on old_distribution and
        # observed_clone_size_distribution

        new_distribution = mixed_distribution(list(zip(new_weights,new_means)))
        total_estimated_clones = round(total_observed_clones / (1. - new_distribution.value_at_clone_size(0)))
        estimated_n_0 = round(total_estimated_clones - total_observed_clones)
        if verbose: print("new estimated n_0\t%i" % estimated_n_0)
        estimated_clone_size_distribution[0] = estimated_n_0 # NB we could try looking at the whole of the estimated distribution, not just estimated_n0, as done here.
        try:
            score_function = 2.*abs(estimated_n_0 - old_estimated_n_0)/float(estimated_n_0 + old_estimated_n_0)
        except ZeroDivisionError:
            score_function = 0.
        if verbose: print("score_function %f" % score_function)
        log_likelihood = calc_log_likelihood_n0_as_parameter(estimated_clone_size_distribution, new_distribution, threshold, total_observed_clones)
        if verbose: print("Log likelihood\t%f\n" % log_likelihood) # Note that this is the log likelihood POST update of estimated_n_0

    dictionary_of_fits[log_likelihood] = new_weights + new_means # Take only self consistent values!
    if count_iterations == iteration_limit:
        with open(file_out,'a') as f:
            f.write("Iteration limit reached - check for only small numbers of clones appearing once or twice in sample\n")
        if verbose: print("Iteration limit reached - check for only small numbers of clones appearing once or twice in sample")
    # double check
    # new_weights = dictionary_of_fits[max(dictionary_of_fits.keys())][:len(list(dictionary_of_fits.values())[0])/2]
    # new_means = dictionary_of_fits[max(dictionary_of_fits.keys())][len(list(dictionary_of_fits.values())[0])/2:]
    new_weights = dictionary_of_fits[max(dictionary_of_fits.keys())][:(int(len(list(dictionary_of_fits.values())[0])/2))]
    new_means = dictionary_of_fits[max(dictionary_of_fits.keys())][int(len(list(dictionary_of_fits.values())[0])/2):]
    new_distribution = mixed_distribution(list(zip(new_weights,new_means)))
    total_estimated_clones = round(total_observed_clones / (1. - new_distribution.value_at_clone_size(0)))
    estimated_n_0 = round(total_estimated_clones - total_observed_clones)

    with open(file_out,'a') as f:
        f.write("\n".join([
                    'n_obs %s' % str(total_observed_clones),
                    'weights = %s' % str(new_distribution.weights),
                    'means = %s' % str(new_distribution.means),
                    'log likelihood = %s' % str(log_likelihood),
                    'estimated_n0 = %s\n\n' % str(estimated_n_0)
                    ])
                )

    if verbose:
        print("Completed iteration of clone sizes")
        print("Fitted weights\t%s" % new_distribution.weights)
        print("Fitted means\t%s" % new_distribution.means)
        print("Final estimated_n0\t%i" % estimated_n_0)
        print("Log likelihood\t%f" % log_likelihood)
        print("Observed clones\t%f" % sum(observed_clone_size_distribution.values()))
        print("Observed clones + estimated clones\t%f" % (sum(observed_clone_size_distribution.values()) + estimated_n_0))
        print("=== END FIT ===\n")

    return new_distribution.weights + new_distribution.means, log_likelihood, estimated_n_0


def reconstruct_overall_distribution(file_in, file_out, total_clones=None, weights = 0., means = 0. ):
    """
    Given an observed sample distribution, returns:
    weights        : a list of weights for the spikes that best describe overall distribution
    means          : a list of means for the spikes that best describe the overall distribution
    n0             : the integer number of missing species
    log-likelihood : the float log-likelihood of this overall distribution, given the observed sample
    time elapsed   : in seconds
    """

    # for calculating time elapsed
    time_0 = time()

    # Read in and format data
    observed_clone_size_distribution = read_observed_clone_size_distribution_v2(file_in, clone_distribution_in_file, threshold) # NB clones larger than threshold never enter fit
    observed_clone_size_distribution = dict((key, value) for key, value in observed_clone_size_distribution.items() if value != 0) # remove zeroes
    
    # set effective_number_of_data_points for AICc
    no_observed_clone_sizes = len(list(observed_clone_size_distribution.keys()))
    effective_number_of_data_points = min(aicc_multiple*no_observed_clone_sizes, threshold)

    # initialize final parameters (will be returned)
    final_weights = []
    final_means = []
    final_n0 = None
    final_log_likelihood = 1.
    final_no_spikes = 0.

    # Initialise starting weights and means. For the first round of fitting this should replicate the scan for two spikes. Initial mean depends only on the observed data:
    initial_mean = sum(clone_size * no_clones for clone_size, no_clones in list(observed_clone_size_distribution.items())) / float(sum(observed_clone_size_distribution.values()))

    starting_weights = [1.]
    starting_means = [initial_mean] # the list of starting points for means
    number_of_parameters = 1

    total_observed_clones = float(sum(observed_clone_size_distribution.values())) # NB this is used as a global variable - be careful!
    new_distribution = mixed_distribution( list(zip(starting_weights, starting_means)) )
    
    # print(new_distribution.value_at_clone_size(12))
    total_estimated_clones = round(total_observed_clones/(1.-new_distribution.value_at_clone_size(0)))
    # print(total_estimated_clones)
    # initialize estimated_n0
    estimated_n_0 = total_estimated_clones - total_observed_clones
    print(total_estimated_clones, total_observed_clones)
    estimated_clone_size_distribution = defaultdict(int)
    estimated_clone_size_distribution[0] = estimated_n_0


    for key in observed_clone_size_distribution:
        estimated_clone_size_distribution[key] = observed_clone_size_distribution[key]
    log_likelihood = calc_log_likelihood_n0_as_parameter(estimated_clone_size_distribution, new_distribution, threshold, total_observed_clones)


    AIC = 2. * (number_of_parameters - log_likelihood)

    if no_observed_clone_sizes > 2:
        AICc = calculate_AICc( AIC, number_of_parameters, no_observed_clone_sizes )
        too_few_clone_sizes = False
    else:
        AICc = float('inf')
        too_few_clone_sizes = True

    AICc_old = float('inf')
    AIC_old = float('inf')

    with open(file_out,'w') as f:
        f.write("\n".join([
                    '# %s' % datetime.now().strftime('%c'),
                    # double check 
                    #'# python %s' % join(argv),
                    '# python %s' % ''.join(argv),
                    'observed_clone_size_distribution = %s' % dict(observed_clone_size_distribution),
                    'total_clones = %s' % total_clones,
                    'weights = 0.',
                    'means = 0.',
                    'initial_scale_factors = %s' % initial_scale_factors,
                    'test_weights_list = %s\n\n' % test_weights_list
                    ])
                )

    minimum_observed_clone_size = min(key for key in observed_clone_size_distribution if observed_clone_size_distribution[key] != 0)

    zero_in_fitted_weights = False

    # n0 is estimated above with the starting mean from data. Now we reinitialise starting weights
    starting_means = []
    starting_weights = []

    while (((AICc < AICc_old) or too_few_clone_sizes) and (number_of_parameters < parameter_limit) and (not zero_in_fitted_weights) and (1e-6 not in starting_means)):

        list_of_fits = [] # The list of fits will have one element for each scan starting point

        for initial_scale_factor in initial_scale_factors:

            # The next loop will add a new weight 
            if starting_weights == []:
                weights_to_loop_over = [1.]
            else:
                weights_to_loop_over = test_weights_list

            for test_weight in weights_to_loop_over:

                starting_means_temp = [ initial_scale_factor * initial_mean ] + starting_means
                starting_weights_temp = [test_weight] + [weight*(1. - test_weight) for weight in starting_weights]

                end_params, log_likelihood, estimated_n_0 = MLE_fit_v2(file_out, threshold, starting_weights_temp, starting_means_temp, observed_clone_size_distribution)

                # An element of list_oF_fits contains the following elements:
                # fit[0] : a list of starting_weights+starting_means
                # fit[1] : a list of fitted end weights+means
                # fit[2] : the log likelihood of the fit
                # fit[3] : estimated n0 for the fit
                list_of_fits.append((starting_weights_temp + starting_means_temp, end_params, log_likelihood, estimated_n_0))

        sorted_fits = sorted(list_of_fits, key = lambda my_list: my_list[2], reverse = True) # sort by log_likelihood
        number_of_spikes = len(sorted_fits[0][0])/2 # This is the length of starting_weights+starting_means in the best fit, divided by 2
        # sorted_means = [fit[0][number_of_spikes:] for fit in sorted_fits]
        sorted_means = [fit[0][int(number_of_spikes):] for fit in sorted_fits] # This takes the starting means for the best fit 
        min_means = [min(m) for m in sorted_means] # This takes the smallest amongst these starting means
        if min_means[0] <= initial_scale_factor * initial_mean: # This checks to see if the smallest starting mean was the mean that was added
            average_start_parameters = sorted_fits[0][0]
            # average_start_parameters[number_of_spikes] = min_means[0] / 2.
            average_start_parameters[int(number_of_spikes)] = min_means[0] / 2.
            # end_params, log_likelihood, estimated_n_0 = MLE_fit_v2(file_out, threshold, average_start_parameters[:len(average_start_parameters)/2], average_start_parameters[len(average_start_parameters)/2:], observed_clone_size_distribution)
            end_params, log_likelihood, estimated_n_0 = MLE_fit_v2(file_out, threshold, average_start_parameters[:int(len(average_start_parameters)/2)], average_start_parameters[int(len(average_start_parameters)/2):], observed_clone_size_distribution)
            list_of_fits.append((average_start_parameters, end_params, log_likelihood, estimated_n_0))
            if verbose: print('SMALLER SPIKE TRIGGERED')

        sorted_fits = sorted(list_of_fits, key = lambda my_list: my_list[2], reverse = True) # sort by log_likelihood

        average_start_parameters = [(p1+p2)/2. for p1,p2 in zip(sorted_fits[0][0], sorted_fits[1][0])] # average best and second best fits
        # end_params, log_likelihood, estimated_n_0 = MLE_fit_v2(file_out, threshold, average_start_parameters[:len(average_start_parameters)/2], average_start_parameters[len(average_start_parameters)/2:], observed_clone_size_distribution)
        end_params, log_likelihood, estimated_n_0 = MLE_fit_v2(file_out, threshold, average_start_parameters[:int(len(average_start_parameters)/2)], average_start_parameters[int(len(average_start_parameters)/2):], observed_clone_size_distribution)
        list_of_fits.append((average_start_parameters, end_params, log_likelihood, estimated_n_0))

        noise_test_ratio = 0.
        # noise_test_ratio compares the expected contribution to the sample of the smallest fitted clones
        # to the noise in the sample from remaining clones
        while noise_test_ratio <= 3.:
            max_log_likelihood = max(fit[2] for fit in list_of_fits)
            log_likelihoods = [element[2] for element in list_of_fits]
            max_index = log_likelihoods.index(max(log_likelihoods))

            fitted_parameters = list_of_fits[max_index][1]

            # Set up starting parameters for the next round of fitting:
            # starting_weights = fitted_parameters[:len(fitted_parameters)/2]
            # starting_means = fitted_parameters[len(fitted_parameters)/2:]
            starting_weights = fitted_parameters[:int(len(fitted_parameters)/2)]
            starting_means = fitted_parameters[int(len(fitted_parameters)/2):]
            sorted_params = sorted(zip(starting_weights, starting_means), key = lambda x: x[1])
            sorted_weights, sorted_means = [[x[i] for x in sorted_params] for i in [0,1]]
            if (0. in sorted_weights) and (len(sorted_weights) == 2): break
            if len(sorted_weights) > 1: # This applies on the second and subsequent interations, when there is more than one spike
                noise_test_ratio = sorted_weights[0]*poisson_pmf(sorted_means[0], minimum_observed_clone_size) / sqrt(sum( [weight * poisson_pmf(mean, minimum_observed_clone_size) for weight, mean in zip(sorted_weights[1:], sorted_means[1:]) ] ) / (total_observed_clones + estimated_n0) )
            else:
                noise_test_ratio = 4. # This applies on the first iteration, when there is only a single spike
            if noise_test_ratio <= noise_test_ratio_threshold:
                del list_of_fits[max_index]
                if len(list_of_fits) == 0:
                    max_log_likelihood = -float('inf')
                    break

        if 0. in fitted_parameters:
            zero_in_fitted_weights = True # flag to stop further iterations
            filtered_weights = [w for w in sorted_weights if w != 0.] # Remove zero from parameters
            filtered_means = [sorted_means[i] for i in range(len(sorted_weights)) if sorted_weights[i] != 0.]
            fitted_parameters = filtered_weights + filtered_means

        AIC_old = AIC
        AICc_old = AICc

        number_of_parameters = len(fitted_parameters) - 1
        AIC = 2.*(number_of_parameters - max_log_likelihood)

        if ((effective_number_of_data_points - number_of_parameters - 1.) > 0.):
            AICc = calculate_AICc(AIC, number_of_parameters, effective_number_of_data_points)
        else:
            AICc = float('inf')

        if 0. in fitted_parameters:
            AICc = float('inf')
            AIC = float('inf')

        if (AICc < AICc_old) or too_few_clone_sizes:
            with open(file_out, 'a') as f: # if you change this to 'w' the following is the only thing that gets output
                f.write('\n===========================================\n')
                if number_of_parameters >= parameter_limit:
                    f.write('parameter_limit = %s # REACHED MAX NUMBER OF PARAMETERS\n' % str(parameter_limit))
                initial_parameters, fitted_parameters, log_likelihood, estimated_n0 = list_of_fits[max_index]
                f.write("\n".join([
                            'initial parameters = %s' % initial_parameters,
                            'fitted parameters = %s' % fitted_parameters,
                            'estimated n0 = %s' % estimated_n0,
                            'AIC = %s' % AIC,
                            'AICc = %s' % AICc,
                            'log likelihood = %s' % log_likelihood,
                            '===========================================\n\n'
                            ])
                        )
            too_few_clone_sizes = False
            final_no_parameters = len(fitted_parameters)/2
            # final_weights = fitted_parameters[:final_no_parameters]
            # final_means = fitted_parameters[final_no_parameters:]
            final_weights = fitted_parameters[:int(final_no_parameters)]
            final_means = fitted_parameters[int(final_no_parameters):]
            final_n0 = estimated_n0
            final_log_likelihood = log_likelihood
    # Carla: added if statement
    if final_n0 == None: final_n0=0
    time_elapsed = time()-time_0
    # return final_weights, final_means, final_n0, observed_clone_size_distribution, total_clones, final_log_likelihood, time_elapsed
    return final_weights, final_means, int(final_n0), observed_clone_size_distribution, total_clones, final_log_likelihood, time_elapsed
# CARLA: CHANGE BACK?


## 1.2 Functions for error-bar fits

In [ ]:
# 1.2 functions for error-bar fits ==========================================


def read_sample_fit(MLE_filename, return_threshold = False):

    with open(MLE_filename, 'rU') as MLE_file:
        try:
            in_best_fit = False
            in_start_params = True
            for line in MLE_file:
                # read parameters from the start of the file...
                if line == '\n':
                    in_start_params = False
                    continue
                # below was originally written for python 2
                # try: left_side, right_side = list(map(strip, split(line, "=", 1)))
                try: left_side, right_side = list(map(strip, line.split("=", 1)))
                except: continue
                if in_start_params:
                    if not line.startswith('#'):
                        # below was originally written for python 2 
                        # line = split(line, '#')[0] 
                        line = line.split('#')[0] ## Is this necessary?
                        if left_side == "parent_population_size": 
                            parent_population_size = literal_eval(right_side)
                        elif left_side == "total_clones":
                            total_clones = literal_eval(right_side)
                        elif left_side == "sample_size": 
                            sample_size = literal_eval(right_side)
                        elif left_side == "weights": 
                            weights = literal_eval(right_side)
                        elif left_side == "means": 
                            means = literal_eval(right_side)
                        elif left_side == "observed_clone_size_distribution": 
                            observed_clone_size_distribution = literal_eval(right_side)
                        elif left_side == "total_observed_clones":
                            total_observed_clones = literal_eval(right_side)
                        elif left_side == "initial_scale_factors":
                            initial_scale_factors = literal_eval(right_side)
                        elif left_side == "test_weights":
                            test_weights = literal_eval(right_side)
                elif line.startswith('='):
                    in_best_fit = not in_best_fit
                # ...and the best-fit parameters from later in the file
                elif in_best_fit:
                    if left_side == "initial parameters":
                        initial_parameters_try = literal_eval(right_side)
                    elif left_side == "fitted parameters":
                        fitted_params_try = literal_eval(right_side)
                    elif left_side == "estimated n0":
                        estimated_n0_try = literal_eval(right_side)
                        if not 1e-6 in fitted_params_try:
                            initial_parameters = initial_parameters_try
                            fitted_params = fitted_params_try
                            estimated_n0 = estimated_n0_try

            observed_threshold = max(observed_clone_size_distribution.keys())

            # set some defaults if they were not read in file
            if 'observed_clone_size_distribution' not in locals():
                if verbose: print('warning: %s does not contain an observed_clone_size_distribution. Skipping...' % MLE_filename)
                return "Bad fit", None, None
            if 'sample_size' not in locals():
                sample_size = sum([key*observed_clone_size_distribution[key] for key in observed_clone_size_distribution])
            if 'total_observed_clones' not in locals():
                total_observed_clones = sum(observed_clone_size_distribution.values())
            if 'parent_population_size' not in locals():
                parent_population_size = 1e9

            # get start weight and calculate start mean
            start_weight = initial_parameters[0]
            initial_mean_norm = float(sum([observed_clone_size_distribution[i] for i in list(observed_clone_size_distribution.keys()) if i <= observed_threshold]))
            initial_mean = sum([i*observed_clone_size_distribution[i] for i in observed_clone_size_distribution if i <= observed_threshold]) / initial_mean_norm
            start_mean = initial_parameters[len(initial_parameters)/2] / initial_mean
            start_mean = int(10*start_mean)/10.

            # return sample_from_parent objects
            number_of_spikes = len(fitted_params)/2
            fit_weights = fitted_params[:number_of_spikes]
            fit_means = fitted_params[number_of_spikes:]
            if return_threshold:
                return observed_threshold, sample_from_parent(weights, means, sample_size, total_clones, parent_population_size, total_observed_clones, estimated_n0, fit_weights, fit_means), start_weight, start_mean, observed_clone_size_distribution
            else:
                return sample_from_parent(weights, means, sample_size, total_clones, parent_population_size, total_observed_clones, estimated_n0, fit_weights, fit_means), start_weight, start_mean, observed_clone_size_distribution

        except ValueError as e:
            if verbose: 
                if very_verbose: 
                    print(e)
                    print(print_exc(file=stdout))
                print('%s: warning: Nan in fit' % MLE_filename)
            return 'Nan in fit', 0, 0, None


def sample_from_model(weights, means, sample_size, total_clones, return_unobserved_clones = False):
    """
    Returns the distribution of a given sample_size most likely to be
    observed, given weights, means, sample_size, and total_clones.
    Compare to output_resample, which I think does exactly the same
    thing, just with a different input.
    Note total_clones is the total number of clones in the *parent*;
    sample_size is the desired sample size of the output.
    """
    sample_size = int(sample_size) # in units of cells
    total_clones = int(total_clones) # Robust to user input as float

    # scale means to sample_size
    scaling_factor = sample_size / (total_clones * sum(w*m for w, m in zip(weights, means)))
    means = [m*scaling_factor for m in means]

    sample_distribution = mixed_distribution( list(zip(weights, means)) )
    observed_clone_size_distribution = {}
    clone_size = 1
    total_observed_clones = 0
    unobserved_clones = total_clones * sample_distribution.value_at_clone_size(0)
    clone_limit = total_clones - unobserved_clones - 1 # When we observe this many clones we can stop looking

    while ( total_observed_clones < clone_limit ):
        new_clones = total_clones * sample_distribution.value_at_clone_size(clone_size)
        observed_clone_size_distribution[clone_size] = int(round(new_clones))
        total_observed_clones += new_clones
        clone_size += 1

    if return_unobserved_clones: return observed_clone_size_distribution, unobserved_clones
    else: return observed_clone_size_distribution


def qD_from_parameters(weights, means, total_estimated_clones, q):
    """
    Used by qD_from_parameters_and_observed_distribution.
    Should add in an estimate from the true distribution for values
    above the upper limit.
    This is slightly awkward, because I am generating the results
    directly from the parameters.
    I need to calculate the observed clone size distribution, work out
    out which clones are larger than the observed cut off and then add
    these to the reconstructed parent distribution.
    """
    zipped_weights_and_means = list(zip(weights, means))
    mean_norm = 1.0 / ( sum(weight*mean for weight,mean in zipped_weights_and_means) * total_estimated_clones )
    # This is one way to think of it; another is as 1. / the total
    # number of cells, in arbitrary units (e.g., units of some
    # fraction of a cell)

    if q == 0.0:
        return total_estimated_clones

    elif q == 1.0:
        qD = -sum(weight * (mean * mean_norm) * log(mean * mean_norm) for weight, mean in zipped_weights_and_means)
        qD = exp(qD * total_estimated_clones)

    elif q == float('inf'):
        qD = 1.0 / max( (mean * mean_norm) for mean in means )

    else:
        qD = sum(weight * (mean * mean_norm)**q for weight, mean in zipped_weights_and_means)
        qD = qD * total_estimated_clones
        qD = qD ** (1.0/(1.0 - q))

    return qD


def qD_from_parameters_and_observed_distribution(weights, means, observed_clone_size_distribution, observed_threshold, q):
    """
    Returns qD. Total_estimated_clones to be passed to this function
    should NOT be the sum of observed clones + estimated_0. It should
    be the sum of all clones arising from the fit parameters. This
    version incorporates various speed ups over version 3.
    """
    max_clone_size = max(observed_clone_size_distribution.keys())
    fitted_distribution = mixed_distribution( list(zip(weights,means)) )

    obs_small_clones = sum( observed_clone_size_distribution[x] for x in range(1, observed_threshold) if x in observed_clone_size_distribution )
    distribution_norm = obs_small_clones / sum( fitted_distribution.value_at_clone_size(x) for x in range(1, observed_threshold) )

    total_estimated_clones_obs = distribution_norm # includes estimated_n0
    for x in range(1,observed_threshold):
        if x not in observed_clone_size_distribution:
            observed_clone_size_distribution[x] = 0

    fit_large_clones = {}
    unfitted_clones = {}
    # NB in the sum below, only take keys that are already in
    # observed_clone_size_distribution. Otherwise, as a defaultdict it
    # will create zero entries.
    for clone_size in range(observed_threshold, max(observed_clone_size_distribution.keys())+1):
        if clone_size in list(observed_clone_size_distribution.keys()):
            if clone_size > max_clone_size_for_which_factorials_can_be_converted_to_float: fit_large_clones[clone_size] = 0. # factorials get very big, very fast. No point in trying to calculate beyond a certain point (that is presumably far lower than the max here)
            else: fit_large_clones[clone_size] = distribution_norm * fitted_distribution.value_at_clone_size(clone_size)
            diff = observed_clone_size_distribution[clone_size] - fit_large_clones[clone_size]
            if diff > 0.:
                unfitted_clones[clone_size] = diff

    aug_total_estimated_clones = float( total_estimated_clones_obs + sum(unfitted_clones.values()) )
    scale_weights = (total_estimated_clones_obs / aug_total_estimated_clones)
    weights = [weight*scale_weights for weight in weights]
    aug_weights = weights + [(unfitted_clones[clone_size] / aug_total_estimated_clones) for clone_size in list(unfitted_clones.keys())]
    aug_means = means + [clone_size for clone_size in list(unfitted_clones.keys())]

    qD = qD_from_parameters(aug_weights, aug_means, aug_total_estimated_clones, q)

    if verbose: 
        print('Number of unfitted clones: %i' % len(unfitted_clones))
        print('aug_weights (should sum to 1.): %s' % sum(aug_weights))

    return qD


def make_error_bar_envelope(samples, q):
    """
    This function considers all examples of fits at constant true
    total clone and returns:
        
    error_envelope_x_vals
    error_envelope_y_vals
        
    The keys of error_envelope_x_vals and error_envelope_x_vals are
    the true qD of the samples which we are using in our error bar
    fits.
        
    For error_envelope_x_vals at a fixed true qD the values are an
    ordered list of sample sizes.
        
    For error_envelope_y_vals at a fixed true qD the values are
    max(abs(log(fitted_D / true_D))) at the sample size given by the
    corresponding error_envelope_x_vals entry. The max is taken over
    steepnesses (that is, distribution shape).
        
    abs_deviations
        
    a list of (sample_size, abs(log(fitted_D / true_D)) pairs.
    """
    error_envelope_x_vals = {}
    error_envelope_y_vals ={}
    abs_deviations = []

    for sample in samples:
        observed_clone_size_distribution = sample_from_model(
            sample.weights, sample.means, sample.sample_size, 
            sample.total_clones
            )
        if observed_clone_size_distribution == {}: # if empty, continue (adds no information)
            continue
        fitted_D = qD_from_parameters_and_observed_distribution(
            sample.fitted_weights, 
            sample.fitted_means, 
            observed_clone_size_distribution, 
            30, 
            q
            )
        true_D = qD_from_parameters(sample.weights, 
                                    sample.means, 
                                    sample.total_clones, 
                                    q
                                    )
        if fitted_D != 0. and true_D != 0.:
            abs_deviations.append(
                (sample.sample_size, abs( log(fitted_D / true_D) ), true_D)
                )

    abs_deviations = [d for d in abs_deviations if not isnan(d[1])]

    true_Ds = sorted(list(set([abs_dev[2] for abs_dev in abs_deviations])))

    for true_D in true_Ds:
        sorted_dev = [dev for dev in abs_deviations if dev[2] == true_D]
        sorted_dev = sorted(sorted_dev, key = lambda x: x[0])
        x_vals = []
        y_vals = []
        max_y_val = 0.
        for val in sorted_dev[::-1]:
            if val[1] > max_y_val:
                max_y_val = val[1]
                if x_vals != [] and val[0] == x_vals[-1]:
                    y_vals[-1] = val[1]
                else:
                    x_vals.append(val[0])
                    y_vals.append(val[1])

        x_vals = x_vals[::-1]
        y_vals = y_vals[::-1]

        # For purposes of interpolation, set the error at 0 sample size equal to the error at smallest available sample size
        if x_vals[0] != 0:
            x_vals = [0] + x_vals
            y_vals = [y_vals[0]] + y_vals

        error_envelope_x_vals[true_D] = x_vals
        error_envelope_y_vals[true_D] = y_vals

    return error_envelope_x_vals, error_envelope_y_vals, abs_deviations


def make_error_bar_file(error_bar_fits_directory, precomputed_error_bar_file, qs):

    time_0 = time()

    # Filehandling
    if isdir(error_bar_fits_directory):
        if not error_bar_fits_directory.endswith('/'):
            error_bar_fits_directory += '/'
    else:
        print("error: %s is not a directory. To make error-bar files, file_in must be a directory that contains the error bar fits. exiting..." % error_bar_fits_directory)
        exit()

    if isfile(precomputed_error_bar_file):
        print("error: precomputed error-bar file %s already exists. exiting..." % precomputed_error_bar_file)
        exit()
    MLE_filenames = listdir(error_bar_fits_directory) # This directory should contain the fits
    MLE_filenames = [i for i in MLE_filenames if i.endswith(".txt")]

    # Read in fitted data for error bars
    fitted_samples = {} # dict[filename] = sample_from_parent objects
    for MLE_filename in MLE_filenames:
        if verbose: print('processing %s' % MLE_filename)
        try:
            fitted_sample, start_weight, start_mean, observed_clone_size_distribution = read_sample_fit(error_bar_fits_directory + MLE_filename)
        except UnboundLocalError:
            if verbose: print('warning: invalid fit: %s skipping...' % MLE_filename)
            continue
        if fitted_sample not in ('Nan in fit', 'Bad fit'):
            fitted_samples[MLE_filename] = fitted_sample

    # actually make error bars and write to file
    z = [fitted_samples[MLE_filename] for MLE_filename in list(fitted_samples.keys())]
    # The keys of error_envelope_x_vals and error_envelope_x_vals are
    # the true 0D of the samples which we are using in our error bar
    # fits. For error_envelope_x_vals at a fixed true 0D the values
    # are an ordered list of sample sizes. For error_envelope_y_vals
    # at a fixed true 0D the values are max(abs(log(fitted_D /
    # true_D))) at the sample size given by the corresponding
    # error_envelope_x_vals entry. The max is taken over steepnesses
    # (that is, distribution shape).
    #
    # Note that error_bar_devs has a default value of q = 1.
    for q in qs:
        error_envelope_x_vals, error_envelope_y_vals, abs_deviations = make_error_bar_envelope(z, q)
        with open(precomputed_error_bar_file, 'a') as f:
            f.write("\n".join([
                        'error_envelope_x_vals_%s = %s' % (q, error_envelope_x_vals),
                        'error_envelope_y_vals_%s = %s' % (q, error_envelope_y_vals),
                        'abs_deviations_%s = %s\n' % (q, abs_deviations)
                        ])
                    )
        if verbose: print("%sD:\tdone (%s)" % (q, precomputed_error_bar_file))

    time_elapsed = time()-time_0
    return time_elapsed


## 1.3 Functions for power tables

In [ ]:
# 1.3 functions for power tables ======================================

def read_error_bar_file(error_bar_file, q=0):
    """
    error_bar_file should be a text file, as generated by make_error_bar_file
    containing values for error_envelope_x_vals and error_envelope_x_vals
    """
    with open(error_bar_file,'rU') as f:
        for line in f:
            if line.startswith("error_envelope_x_vals_%s" % q):
                # below was originally written for python 2
                # error_envelope_x_vals = literal_eval( strip(split(line, "=")[1]) )
                error_envelope_x_vals = literal_eval(  (line.split("=")).strip([1]))
            elif line.startswith("error_envelope_y_vals_%s" % q):
            	# below was originally written for python 2
            	# error_envelope_y_vals = literal_eval( strip(split(line, "=")[1]) )
                error_envelope_y_vals = literal_eval( (line.split("=")).strip([1]))

    return error_envelope_x_vals, error_envelope_y_vals


def eb(sample_size, true_total_clone, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, target):
    """
    This returns the log of high error with the given parameters.
    The actual error will be exp(eb_val) - true_total_clone
    exp is because error_envelope_y_vals made by compare_MLE_alpha_v7.py contains log(fitted/true) values.
    """
    available_total_clone_sizes = sorted(error_envelope_x_vals.keys())
    for i in range(len(available_total_clone_sizes)):
        if available_total_clone_sizes[i] >= true_total_clone: break

    denom = float(available_total_clone_sizes[i] - available_total_clone_sizes[i-1])
    weight_1 = (available_total_clone_sizes[i] - true_total_clone) / denom
    weight_0 = (true_total_clone - available_total_clone_sizes[i-1]) / denom

    interpolated_1 = interp(sample_size, error_envelope_x_vals[available_total_clone_sizes[i-1]], error_envelope_y_vals[available_total_clone_sizes[i-1]])
    interpolated_0 = interp(sample_size, error_envelope_x_vals[available_total_clone_sizes[i]], error_envelope_y_vals[available_total_clone_sizes[i]])

    eb_val = weight_1 * interpolated_1 + weight_0 * interpolated_0

    return number_of_error_bars*(exp(eb_val) - 1.) - target


def bisect_eb(eb, true_total_clone, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, target):
    """
    This finds a root of number_of_error_bars*(exp(eb) - 1.) - target = 0.
    by iterated bisection.
    """
    low = 0
    high = true_total_clone*10
    eb_low = eb(low, true_total_clone, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, target)
    if eb_low < 0.:
        available_total_clone_sizes = sorted(error_envelope_x_vals.keys())
        for i in range(len(available_total_clone_sizes)):
            if available_total_clone_sizes[i] >= true_total_clone:
                break
        return sorted(error_envelope_x_vals[available_total_clone_sizes[max(0,i-1)]])[1]

    eb_high = eb(high, true_total_clone, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, target)
    assert eb_low * eb_high <= 0.
    while high - low > 1:
        midpoint = (low + high) / 2.
        eb_midpoint = eb(midpoint, true_total_clone, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, target)
        if eb_low * eb_midpoint > 0.:
            low = midpoint
        else:
            high = midpoint
    return midpoint


def get_sample_size(number_of_clones, fold_difference, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars):
    """
    This returns the number of cells in a sample that produce the an
    error bar of max_error_bar for a given number_of_clones in the
    parent population.
    
    This is the inverse function of the calculation performed by
    error_bar_on_fit_qD
    
    Delta = (fold_difference - 1.0) * number_of_clones
    """
    target = fold_difference - 1.
    sample_size = bisect_eb(eb, number_of_clones, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, target)
    return sample_size


def get_sampling_limit(number_of_clones, min_number_of_doublets, fraction_small_clones):
    """
    This function returns the minimum number of cells required to
    reliably see doublets and triplets.
    
    The number of possible pairs of cells is sample_size**2/2
    
    Expected doublets = p * sample_size**2/2
    
    However, we need an effective sample size, given by the sample_size * fraction_of_small_clones
    where fraction_of_small_clones is the fraction of cells in small clones.    
        
    Expected doublets = p * (sample_size * fraction_of_small_clones)**2/2
        
    Where p is the probability that a pair of cells is are from the
    same clone, given that both cells are from small clones.
    
    Now estimate p = 1/number_of_small_clones, which is approximately
    1/number_of_clones.
    
    Expected doublets = (sample_size * fraction_of_small_clones)**2/ (2 * number_of_clones)
    We want to expect at least min_number_of_doublets.
        
    rearranging:
        
    sample_size = sqrt(2 * number_of_clones * min_number_of_doublets) / fraction_of_small_clones
    
    This is our sampling limit.
    """
    return int(round( sqrt(2.*min_number_of_doublets*number_of_clones) / fraction_small_clones ))


def get_minimum_cells(list_of_number_of_clones, list_of_fold_differences, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, min_number_of_doublets, fraction_small_clones):
    minimum_cells = {}
    for fold_difference in list_of_fold_differences:
        for number_of_clones in list_of_number_of_clones:
            # Calculate the minimum number of cells required to reliably see a fold.
            fold_limit = get_sample_size(number_of_clones, fold_difference, error_envelope_x_vals, error_envelope_y_vals,  number_of_error_bars)
            # Calculate the minimum number of cells required to reliably see doublets and triplets
            sampling_limit = get_sampling_limit(number_of_clones, min_number_of_doublets, fraction_small_clones)
            # Take the more stringent result
            minimum_cells[number_of_clones, fold_difference] = int(round(max(fold_limit, sampling_limit)))

    return minimum_cells


def output_table(list_of_number_of_clones, list_of_fold_differences, error_bar_file, number_of_error_bars, min_number_of_doublets, fraction_small_clones, minimum_cells, q, file_output = None):

    lines = []

    # Record variables used
    lines.append('# %s' % datetime.now().strftime('%c'))
    lines.append('# python %s' % ' '.join(argv))
    lines.append('error_bar_file = %s' % error_bar_file)
    lines.append('number_of_error_bars = %s' % number_of_error_bars)
    lines.append('min_number_of_doublets = %s' % min_number_of_doublets)
    lines.append('fraction_small_clones = %s' % fraction_small_clones)
    lines.append('q = %s\n' % q)

    # make header line
    line = "\t" + "\t".join([str(int(number_of_clones)) for number_of_clones in list_of_number_of_clones])
    lines.append(line)

    # iterate over lines
    for fold_difference in list_of_fold_differences:

        # iterate over cols within line
        line = str(fold_difference) + "\t" 
        line += "\t".join([str(minimum_cells[number_of_clones, fold_difference]) for number_of_clones in list_of_number_of_clones])
        lines.append(line)
        
    # output lines
    lines = "\n".join(lines)+"\n"
    if file_output:
        if isfile(file_output):
            print("warning: power table file %s already exists. Printing to stdout and exiting..." % file_output)
            print(lines)
            exit()
        with open(file_output, "a") as f:
            f.write(lines)
    elif verbose: print(lines)
    return


## 1.4 Functions for d-number tables

In [ ]:
# 1.4 functions for D-number tables ======================================

def qD_from_distribution(H, q):
    """
    Returns the qD given a distribution in the form of a dictionary H = { clone_size : clone number }
    """
    # H is a dictionary-like object
    N = float(sum( k*v for k, v in list(H.items()) )) # normalization factor
    H = dict( (k/N, v) for k, v in list(H.items()) ) # normalize key entries by N, making them probabilities
    if q == float("inf"): return 1./max(H.keys())
    elif q == 1.: return exp( -sum( v*k*log(k) for k, v in list(H.items()) if v != 0. ) )
    else: return sum( v*k**q for k, v in list(H.items()) )**(1./(1.-q))


def error_bar(sample_size, true_total_clone, error_envelope_x_vals_q, error_envelope_y_vals_q):
    """
    The function error_bar(sample_size, true_total_clone,
    error_envelope_x_vals, error_envelope_y_vals) returns the value of
    max(abs(fitted_D / true_D)) at the given sample size and given
    total_clones.  Since we only have values at a grid of true_D and a
    grid of sample size, this is calculated by interpolation. This
    will be an interpolation between
        
        error_bar(sample_size, true_total_clone1)
        
    and
        
        error_bar(sample_size, true_total_clone2)
        
    and then between sample sizes.
    """
    available_total_clone_sizes = sorted(error_envelope_x_vals_q.keys())
    if true_total_clone > available_total_clone_sizes[-1] or (true_total_clone < available_total_clone_sizes[0]):
        if verbose: print('Clone size out of range')
        return None
    for i in range(len(available_total_clone_sizes)):
        if available_total_clone_sizes[i] >= true_total_clone:
            break
                                                              
    weight_0 = (available_total_clone_sizes[i] - true_total_clone) / float(available_total_clone_sizes[i] - available_total_clone_sizes[i-1])
    weight_1 = (true_total_clone - available_total_clone_sizes[i-1]) / float(available_total_clone_sizes[i] - available_total_clone_sizes[i-1])

    interp_0 = interp(sample_size, error_envelope_x_vals_q[available_total_clone_sizes[i-1]], error_envelope_y_vals_q[available_total_clone_sizes[i-1]])
    interp_1 = interp(sample_size, error_envelope_x_vals_q[available_total_clone_sizes[i]], error_envelope_y_vals_q[available_total_clone_sizes[i]])
                                                              
    return weight_0*interp_0 + weight_1*interp_1


def abs_error(total_clones, sample_size, error_envelope_x_vals_q, error_envelope_y_vals_q,high_low_sign):
    """
    high_low_sign = +1 will return the upper limit on the number of
    reconstructed clones for 0D = total clones and given sample size.
    Dependence on q (as which D numbers, qD) is embodied in
    error_envelope_x_vals and error_envelope_y_vals. These must be
    calculated for each q, and the appropriate dictionary passed to
    the abs_error function.
    """
    error_bar_result = error_bar(sample_size, total_clones, error_envelope_x_vals_q, error_envelope_y_vals_q)
    if error_bar_result == None: # clone size out of range
        return None
    else: 
        return total_clones * exp(high_low_sign * error_bar_result ) # NB total_observed_clones is the sample size


def error_bar_on_fit_qD(sample_size, reconstructed_qD, error_envelope_x_vals_q, error_envelope_y_vals_q):

    step_size = (reconstructed_qD / 1000.0)

    # calculate lower limit
    n = 1; abs_error_result = float("inf") # initialization
    while abs_error_result >= reconstructed_qD:
        abs_error_result = abs_error((reconstructed_qD - n*step_size), sample_size, error_envelope_x_vals_q, error_envelope_y_vals_q, +1)
        if abs_error_result == None:
            lower_limit = 0.
            break
        n += 1
        lower_limit = reconstructed_qD - (n+1)*step_size

    # calculate upper limit
    n = 1; abs_error_result = 0. # initialization
    while abs_error_result <= reconstructed_qD:
        abs_error_result = abs_error((reconstructed_qD + n*step_size), sample_size, error_envelope_x_vals_q, error_envelope_y_vals_q, -1)
        if abs_error_result == None:
            upper_limit = float("inf")
            break
        n += 1
        upper_limit = reconstructed_qD + (n+1)*step_size

    return lower_limit, upper_limit


def output_table_of_D_numbers(precomputed_error_bar_file, D_number_output_file, infiles, qs = [0., 1., 2., float("inf")], write_to_disk = True ):
    """
    Returns a dictionary of dictionaries: file_q_qD_hash[file] = dict[q] : (obs_qD, recon_qD, recon_lower_limit, recon_qD_upper_limit)
    #
    If no precomputed_error_bar_file is given, then proceeds without error bars
    """
    # read in error-bar envelope variables
    if precomputed_error_bar_file:
        error_envelope_xs = {}
        error_envelope_ys = {}
        # with open(precomputed_error_bar_file,'rU') as f:
        with open(precomputed_error_bar_file) as f:
            for line in f:
            	# below was originally written for python 2
                # left_side, right_side = list(map(strip, split(line, "=")))
                # left_side, right_side = list(map(strip, line.split("=")))
                left_side, right_side = list(map(str.strip, line.split("=")))
                name, q = left_side.rsplit("_",1)
                q = float(q)
                if name == "error_envelope_x_vals":
                    error_envelope_xs[q] = literal_eval(right_side)
                elif name == "error_envelope_y_vals":
                    error_envelope_ys[q] = literal_eval(right_side)

    # initialize
    estimated_n0 = {}  # estimated_n0[infile] = estimated n0 for that file
    fitted_params = {} # fitted_params[infile] = final fitted params for that file
    observed_clone_size_distributions = {} # observed_clone_size_distributions[infile] = observed_clone_size_distribution for that file
    obs_qDs = defaultdict(dict) # obs_qDs[filename] = obs_qD{q: qD}
    observed_threshold = 30 # not recorded in files but needed
    
    # Read in observed_clone_size_distributions, fitted_parameters,
    # and n0s from MLE files (note similarity to read_sample_fit)
    for infile in infiles:
        obs_qD = {}
        # with open(infile,'rU') as MLE_file:
        with open(infile) as MLE_file:
            in_best_fit = False
            in_start_params = True
            for line in MLE_file:
                if line == "\n": 
                    in_start_params = False
                    continue
                # below was originally written for python 2    
                # try: left_side, right_side = list(map(strip, split(line, "=", 1)))
                # try: left_side, right_side = list(map(strip, line.split("=", 1)))
                try: left_side, right_side = list(map(str.strip, line.split("=", 1)))
                except: continue
                if in_start_params:
                    # read in observed_clone_size_distribution and threshold (if present) from the start of the file...
                    if left_side == "observed_clone_size_distribution": # This requires full clone distribution in fit file, including clones that are not explicitly fitted.
                        observed_clone_size_distribution = literal_eval(right_side)
                        observed_clone_size_distributions[infile] = observed_clone_size_distribution
                    elif left_side == "threshold":
                        threshold_in_file = literal_eval(right_side)
                        if not observed_threshold:
                            observed_threshold = threshold_in_file
                        elif observed_threshold != threshold_in_file:
                            print('error: all fit files in one table must use the same fitting threshold. Exiting...')
                            exit()
                elif line.startswith('='):
                    in_best_fit = not in_best_fit
                # ...and the best-fit parameters from later in the file
                elif in_best_fit:
                    if left_side == "fitted parameters":
                        fitted_params_try = literal_eval(right_side)
                    elif left_side == "estimated n0":
                        estimated_n0_try = literal_eval(right_side)
                        if not 1e-6 in fitted_params_try:
                            fitted_params[infile] = fitted_params_try
                            estimated_n0[infile] = estimated_n0_try

        # calculate observed qDs
        for q in qs: 
            obs_qD[q] = qD_from_distribution(observed_clone_size_distribution, q)
        obs_qDs[infile] = obs_qD
        #"""


    # write D-number table
    file_q_qD_hash = defaultdict(lambda : defaultdict(tuple) ) # file_q_qD_hash[file] = dict[q] : (obs_qD, recon_qD, recon_qD-, recon_qD+); for return

    # header line
    header_line = "sample_name\t" + "\t".join(["obs_%sD\test_%sD\test_%sD-\test_%sD+" % (q, q, q, q) for q in qs]) + "\n"
    if write_to_disk:
        with open(D_number_output_file, "w") as f:
            f.write("\n".join([
                        # double check
                        # "# python %s" % join(argv),
                        "# python %s" % ''.join(argv),
                        "# infiles = %s" % infiles,
                        "# observed_threshold = %s" % observed_threshold,
                        "# precomputed_error_bar_file = %s" % precomputed_error_bar_file,
                        header_line
                        ]))

    for infile in infiles:

        print(infile)
        # get sample_size (needed for error bars)
        if infile not in list(fitted_params.keys()): 
            print("Error: bad fit: either Recon failed to produce a good fit (check for a very small number in the fitted parameters), or else the input file was not a Recon output-format file. Skipping...")
            continue
        # weights = fitted_params[infile][:len(fitted_params[infile])/2]
        # means = fitted_params[infile][len(fitted_params[infile])/2:]

        weights = fitted_params[infile][:int(len(fitted_params[infile])/2)]
        means = fitted_params[infile][int(len(fitted_params[infile])/2):]
        observed_clone_size_distribution = observed_clone_size_distributions[infile]
        sample_size = sum(key*observed_clone_size_distribution[key] for key in observed_clone_size_distribution if key < observed_threshold)
        
        # get qD_obs, qD, qD-, qD+ for each q
        outline = [infile]
        for q in qs:
            q = float(q)
            obs_qD = obs_qDs[infile][q]
            recon_qD = qD_from_parameters_and_observed_distribution(weights, means, observed_clone_size_distribution, observed_threshold, q)
            try:
                lower_limit, upper_limit = error_bar_on_fit_qD(sample_size, recon_qD, error_envelope_xs[q], error_envelope_ys[q])
                lower_limit = max(lower_limit, obs_qD) # lower_limit can't be lower than the obs_qD
                upper_limit = 1.0164*upper_limit # (very minor) adjustment based on validation, to achieve 95% confidence internvals
            except: 
                lower_limit = ""
                upper_limit = ""
            outline += [obs_qD, recon_qD, lower_limit, upper_limit]
            file_q_qD_hash[infile][q] = (obs_qD, recon_qD, lower_limit, upper_limit)
                
        outline = list(map(str, outline))
        outline = "\t".join(outline)
        
        # write output to file (one write per infile)
        if write_to_disk:
            with open(D_number_output_file, 'a') as f:
                f.write(outline+"\n")

        if verbose: 
            print("infile:\t%s" % infile)
            print("weights:\t%s" % weights)
            print("means:\t%s" % means)
            print("observed_clone_size_distribution:\t%s" % observed_clone_size_distribution)
            print("sample_size:\t%s" % sample_size)
            print()
        
    return file_q_qD_hash


# 1.4 functions for resmapling ======================

def output_resample(fitfile, outfile, write_to_file = True):
    """
    Given a file that contains a fit (with fitted parameters, etc.),
    which recall describes a reconstructed overall population, this
    function outputs a sample of sample size sample_size that can be
    compared to the initial sample from which the reconstructed
    overall population was made. Also writes this to outfile.
    Note the resample is the same size as the original. This, and that
    it operates directly on a file as opposed to weights and means,
    distinguishes this function from sample_from_model().
    """
    largest_observed_clone, model, start_weight, start_mean, observed_clone_size_distribution = read_sample_fit(fitfile, return_threshold = True)  # see 1.2 above

    observed_clone_size_distribution = defaultdict(int, observed_clone_size_distribution)
    sample_distribution = mixed_distribution(list(zip(model.fitted_weights, model.fitted_means)))
    total_clones = model.total_clones
    total_observed_clones = 0
    unobserved_clones = model.estimated_n0

    if total_clones == None: total_clones = unobserved_clones + model.total_observed_clones

    # When we see this many clones we can stop looking. Equivalent to
    # total_clones - actual_n0. This makes more sense than
    # total_clones - estimated_n0, because that fixes the n0,
    # restricting room for the observed; whereas the uncertainty is in
    # the n0, so we should not restrict room for the observed in this
    # way.
    clone_limit = sum(observed_clone_size_distribution.values()) 

    resampled_clone_size_distribution = {}
    clone_size = 1
    while total_observed_clones < total_clones and clone_size <= largest_observed_clone:
        if clone_size > max_clone_size_for_which_factorials_can_be_converted_to_float: no_clones_at_this_size = 0
        else: no_clones_at_this_size = sample_distribution.value_at_clone_size(clone_size)
        new_clones = total_clones * no_clones_at_this_size
        if clone_size <= threshold: # add back large clones (since they were not fit anyway, and their frequency is well sampled)
            resampled_clone_size_distribution[clone_size] = int(round(new_clones))
        else: resampled_clone_size_distribution[clone_size] = observed_clone_size_distribution[clone_size]
        total_observed_clones += new_clones
        clone_size += 1


    header = "#clone_size\tno_clones"
    outlines = [header]
    resampled_clone_sizes = list(resampled_clone_size_distribution.keys())
    outlines += ["%i\t%i" % (i, resampled_clone_size_distribution[i]) for i in range(1, largest_observed_clone+1) if i in resampled_clone_sizes and resampled_clone_size_distribution[i] != 0]
    if write_to_file:
        with open(outfile, "w") as f: # write to disk
            f.write("\n".join(outlines) + "\n")
    return resampled_clone_size_distribution, observed_clone_size_distribution


## 1.4 Functions for plotting resample

In [ ]:
#  1.5 functions for plotting resample ======================================


def write_html_file(html_file):
    """
    Writes the bare-bones html that calls the javascript that creates the plots in d3.js.
    """
    with open(html_file, "w") as f:
        f.write("""
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>plot_clone_size_distribution</title>
    <link href='http://fonts.googleapis.com/css?family=Roboto' rel='stylesheet' type='text/css'>
    <link href="%s/style.css" rel="stylesheet" type="text/css" />
    <script type="text/javascript" src="%s/d3.js"></script>
  </head>
  <body>
    <script type="text/javascript" src="%s/plot_clone_size_distribution.js">
    </script>
  </body>
</html>
""" % (resource_path, resource_path, resource_path) )
        return


def customize_js(javascript_file, plotfile, x_max="false", y_max="false"):
    txt = open(javascript_file, "r").read()
    txt = txt.replace("X_MAX", str(x_max))
    txt = txt.replace("Y_MAX", y_max)
    txt = txt.replace("SUPPRESS_OBS_ZERO", "false")
    txt = txt.replace('var filename = "test.txt";', 'var filename = "%s";' % plotfile)
    javascript_outfile = javascript_file.replace("_ref", "")
    f = open(javascript_outfile, "w")
    f.write(txt)
    f.close()
    return


def round_up_to_the_nearest(x, unit=10):
    return int(unit * ceil(float(x)/unit))

def plot_resample(fitfile, plotfile, precomputed_error_bar_file, x_max = x_max, y_max = y_max, javascript_file = resource_path + "/" + javascript_file, q=0.):
# def plot_resample(fitfile, plotfile, precomputed_error_bar_file, x_max, y_max, javascript_file = resource_path + "/" + javascript_file, q=0.):
    """
    Note need javascript file plot_clone_size_distribution_ref.js as well as style.css and d3.js in current working directory.
    """
    # make sure plotfile includes full path (so the javascript file knows where to look for it)
    full_path = getcwd()
    # below two lines were originally written for python 2 
    # if not isabs(rsplit(javascript_file, "/", 1)[0]): javascript_file = full_path + "/" + javascript_file
    # if not isabs(rsplit(plotfile, "/", 1)[0]): plotfile = full_path + "/" + plotfile
    if not isabs(javascript_file.rsplit("/", 1)[0]): javascript_file = full_path + "/" + javascript_file
    if not isabs(plotfile.rsplit("/", 1)[0]): plotfile = full_path + "/" + plotfile
    # get datapoints for observed and resampled
    resampled_clone_size_distribution, observed_clone_size_distribution = output_resample(fitfile, None, False)
    # get estimated_n0
    fitted_sample, x, x, x = read_sample_fit(fitfile)
    estimated_n0 = fitted_sample.estimated_n0
    # get error bars on estimated_n0
    file_q_qD_hash = output_table_of_D_numbers(precomputed_error_bar_file, None, [fitfile], [q], False)
    obs_qD, recon_qD, recon_qD_lower_limit, recon_qD_upper_limit = file_q_qD_hash[fitfile][q]
    # apply error bars to estimated_n0
    lower_n0_error_bar = estimated_n0 - (recon_qD - recon_qD_lower_limit)
    lower_n0_error_bar = max(0.7, lower_n0_error_bar) # actually zero; did this to make it appear on the plot
    upper_n0_error_bar = estimated_n0 + (recon_qD_upper_limit - recon_qD)
    # get actual n0
    total_clones = fitted_sample.total_clones
    if total_clones:
        actual_n0 = total_clones - sum(observed_clone_size_distribution.values())
    else: actual_n0 = 0. # suppresses display (since y-axis is in log units)
    actual_n0 = max(actual_n0, 0.) # avoid negatives, which get plotted high
    # get clone sizes
    observed_clone_size_distribution = defaultdict(int, observed_clone_size_distribution) # so looking for a clone size that isn't there returns zero instead of an error
    resampled_clone_size_distribution = defaultdict(int, resampled_clone_size_distribution) # ditto
    clone_sizes = list(resampled_clone_size_distribution.keys()) + list(observed_clone_size_distribution.keys())
    clone_sizes = list(set(clone_sizes)) # uniques the clone_sizes
    clone_sizes = [i for i in clone_sizes if resampled_clone_size_distribution[i] != 0 or observed_clone_size_distribution[i] != 0]
    clone_sizes.sort()

    # make plotfile
    with open(plotfile, "w") as f:
        # write header
        header = "\t".join(["clone_size", "sample", "fit", "lower_limit", "upper_limit"])
        f.write(header+"\n")
        # write clone-size zero
        outline = [0, actual_n0, estimated_n0, lower_n0_error_bar, upper_n0_error_bar]
        outline = list(map(str, outline))
        outline = "\t".join(outline)
        f.write(outline+"\n")
        if verbose: 
            print(header)
            print(outline)
        # write other clone sizes
        for clone_size in clone_sizes:
            if clone_size == 0: continue # we already took care of zero
            outline = [clone_size, observed_clone_size_distribution[clone_size], resampled_clone_size_distribution[clone_size]]
            outline = list(map(str, outline))
            outline = "\t".join(outline)
            f.write(outline+"\n")
            if verbose: print(outline)

    # set x and y limits
    if not x_max: 
        x_max = max( round_up_to_the_nearest(max(clone_sizes), 10), 40 )
    if not y_max: 
        y_max = max(list(observed_clone_size_distribution.values())+list(resampled_clone_size_distribution.values())+[recon_qD_upper_limit])
        if y_max == float('inf'): y_max = max(list(observed_clone_size_distribution.values())+list(resampled_clone_size_distribution.values()))
        y_max = log(y_max, 10)
        y_max = round_up_to_the_nearest(y_max, 1)
        y_max = 10**y_max

    # update javascript
    customize_js(javascript_file, plotfile, str(x_max), str(y_max))

    # the absolute path to the .js and .css must be given in the html DONE
    # the absolute path to the plotfile must be given in plot_clone_size_distribution.js

    # Use wkhtmltopdf to convert the html to PDF and cpdf to crop the PDFs
    pdf_file = fitfile.replace(".txt", ".pdf")    
    html_file = fitfile.replace(".txt", ".html")
    write_html_file(html_file)
    print("  writing %s..." % pdf_file)
    command = "wkhtmltopdf '%s' '%s'" % (html_file, pdf_file)
    # x = getoutput(command)
    x = subprocess.getoutput(command)

    """
    Crop the result for easier use in graphics-layout programs
    (e.g. OmniGraffle). Note the parameters below (x, y, width,
    height, with the origin at the bottom left), were chosen for
    dimension_base = 500 in
    plot_clone_size_distribution_ref.js. Smaller values of
    dimension_base will result in white borders with these parameters.
    """
    command = 'cpdf -crop "38mm 185mm 112mm 95mm" \'%s\' -o \'%s\'' % (pdf_file, pdf_file)
    system(command)

    return


def get_upper_bound(fit_file, D_number_file, N, sample_size):
	"""
	U_max = R * w_min * m_min * N/S
	U_max: upper bound that we seek to calculate
	R:     Recon's (*)upper bound of) the overall species richness estimate (from D_number_file)
	N:     overall population size (from user)
	S:     sample size (from user)
	m_min: smallest clone (from fit_file)
	w_min: weight of smallest clone (from fit_file)
	fit_file is a Recon fit file. We read 
	"""
	#
	# Make sure N and sample_size were passed
	if N == None:
		print("error in get_upper_bound(): Please supply a total population size using the -N option. Exiting...")
		exit()
	if sample_size == None:
		print("error in get_upper_bound(): Please supply a sample size using the -ss or --sample_size option. Exiting...")
		exit()
	#
	# get m_min and w_min
	if not isfile(fit_file):
		print("error in get_upper_bound(): Cannot find fit_file %s . Exiting..." % fit_file)
		exit()
	if not isfile(D_number_file):
		print("error in get_upper_bound(): Cannot find D_number_file %s . Exiting..." % D_number_file)
		exit()
	with open(fit_file) as f:
		txt = f.read()
		final_parameters = txt.split("===========================================")[-2]
		del(txt)
		final_parameters = final_parameters.split("\n")
		for line in final_parameters:
			# below was originally written for python 2
			# try: left_side, right_side = list(map(strip, split(line, "=", 1)))
			# try: left_side, right_side = list(map(strip, line.split("=", 1)))
			try: left_side, right_side = list(map(str.strip, line.split("=", 1)))
			except: continue
			if left_side == "fitted parameters":
				weights_and_means_list = literal_eval(right_side)
		number_of_parameters = len(weights_and_means_list)
		weights = weights_and_means_list[:number_of_parameters/2]
		means = weights_and_means_list[number_of_parameters/2:]
		m_min, w_min = sorted(zip(means, weights))[0] # sorts from smallest to largest m, with associated w
	#
	# get R
	with open(D_number_file) as f:
		for line in f:
			if line.startswith("#"): continue # in case our keyword happens to be in a filename
			if line.startswith("sample_name"):
				# below was originally written for python 2
				# header = split(strip(line))
				# header = (line.strip()).split()
				header = (line.str.strip()).split()
				continue
			if line.split("\t")[0] in fit_file:
				# below was originally written for python 2
				# vals = split(strip(line))
				# vals = (line.strip()).split() 
				vals = (line.str.strip()).split() 
				break
		lookup_hash = {i:j for i, j in zip(header, vals)}
		R = float(lookup_hash["est_0.0D+"])
	#
	# float N and S
	N = float(N)
	S = float(sample_size)
	#
	# calculate U_max
	U_max = R * w_min * m_min * N/S
	#
	return U_max


# Executables

## Sonia/TCRdb

#### 1 - SONIA 1

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_1all.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


##### 1.1 df1-5 standard

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population

elif make_error_bars:
    print("making error-bar file...")
    print(make_error_bar_file(file_in, file_out, qs))

elif make_table_of_D_numbers:
    print("making table of D numbers...")
    output_table_of_D_numbers(precomputed_error_bar_file, file_out, list_of_filenames, qs)

elif make_power_table:
    print("making power table...")
    error_envelope_x_vals, error_envelope_y_vals = read_error_bar_file(file_in, q)
    minimum_cells = get_minimum_cells(list_of_number_of_clones, list_of_fold_differences, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, min_number_of_doublets, fraction_small_clones)
    output_table(list_of_number_of_clones, list_of_fold_differences, file_out, number_of_error_bars, min_number_of_doublets, fraction_small_clones, minimum_cells, q, file_out)

elif resample:
    print("resampling...")
    output_resample(file_in, file_out)

elif make_resample_plot:
    print("plotting resample...")
    plot_resample(file_in, file_out, precomputed_error_bar_file)

elif upper_bound:
	print("calculating strict upper limit U...")
	print(get_upper_bound(fit_file, D_number_file, N, sample_size))


else:
    print("""error: no option given. Options:
-R  --run_recon
-e  --make_error_bars
-p  --make_power_table
-T  --make_table_of_D_numbers
-r  --resample
-x  --make_resample_plot
exiting...
""")
    exit()

running recon on df_fill_in_test.tsv...


/usr/local/lib/python3.9/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


([0.05, 0.0475, 0.045125, 0.04286875, 0.040725312500000006, 0.43387002511694217, 0.20394654742983467, 0.08157861897193386, 0.0027192872990644632, 0.0516664586822248], [0.06342727504755778, 0.002973153517854271, 0.005946307035708542, 0.011892614071417083, 0.023785228142834167, 0.04757045628566833, 0.06342727504755778, 0.142711368857005, 1.3954000510462712, 0.5074182003804623], 51468404, {1: 3552422, 2: 223562, 3: 68451, 4: 31801, 5: 17608, 6: 10986, 7: 7533, 8: 5237, 9: 4003, 10: 3025, 11: 2467, 12: 1959, 13: 1521, 14: 1286, 15: 1097, 16: 891, 17: 757, 18: 693, 19: 584, 20: 538, 21: 414, 22: 379, 23: 346, 24: 303, 25: 273, 26: 259, 27: 218, 29: 200, 28: 195, 30: 177, 31: 164, 32: 148, 33: 116, 35: 113, 34: 109, 37: 97, 36: 96, 38: 79, 39: 77, 40: 75, 42: 68, 41: 66, 44: 64, 45: 51, 43: 49, 46: 48, 47: 39, 48: 39, 50: 38, 57: 35, 52: 34, 49: 33, 51: 30, 54: 29, 55: 29, 56: 29, 53: 27, 60: 26, 63: 22, 68: 22, 62: 19, 69: 18, 58: 17, 61: 17, 59: 16, 75: 16, 66: 16, 71: 15, 65: 14, 70: 13, 

##### 1.2 df1-5 verbose = true

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population

elif make_error_bars:
    print("making error-bar file...")
    print(make_error_bar_file(file_in, file_out, qs))

elif make_table_of_D_numbers:
    print("making table of D numbers...")
    output_table_of_D_numbers(precomputed_error_bar_file, file_out, list_of_filenames, qs)

elif make_power_table:
    print("making power table...")
    error_envelope_x_vals, error_envelope_y_vals = read_error_bar_file(file_in, q)
    minimum_cells = get_minimum_cells(list_of_number_of_clones, list_of_fold_differences, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, min_number_of_doublets, fraction_small_clones)
    output_table(list_of_number_of_clones, list_of_fold_differences, file_out, number_of_error_bars, min_number_of_doublets, fraction_small_clones, minimum_cells, q, file_out)

elif resample:
    print("resampling...")
    output_resample(file_in, file_out)

elif make_resample_plot:
    print("plotting resample...")
    plot_resample(file_in, file_out, precomputed_error_bar_file)

elif upper_bound:
	print("calculating strict upper limit U...")
	print(get_upper_bound(fit_file, D_number_file, N, sample_size))


else:
    print("""error: no option given. Options:
-R  --run_recon
-e  --make_error_bars
-p  --make_power_table
-T  --make_table_of_D_numbers
-r  --resample
-x  --make_resample_plot
exiting...
""")
    exit()

Streaming output truncated to the last 5000 lines.
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.05, 0.0475, 0.045125, 0.04286875000000001, 0.45670528959678125, 0.21468057624193124, 0.08587223049677249, 0.0028624076832257507, 0.05438574598128927] [1.3954000510462712, 0.005946307035708542, 0.011892614071417083, 0.023785228142834167, 0.04757045628566833, 0.06342727504755778, 0.142711368857005, 1.3954000510462712, 0.5074182003804623]
new estimated n_0	32177593
score_function 0.000000
Log likelihood	-2548393.838232

Completed iteration of clone sizes
Fitted weights	[0.05, 0.0475, 0.045125, 0.04286875000000001, 0.45670528959678125, 0.21468057624193124, 0.08587223049677249, 0.0028624076832257507, 0.05438574598128927]
Fitted means	[1.3954000510462712, 0.005946307035708542, 0.01189

In [ ]:
# this is a very low percentage of the assumed amount of clones (6%)
# the log likelihood is super low and not increasing much over time. There needs to be more data for this analysis
100* 3941522/58262035

6.7651636267081985

##### 1.3 df1-5 very verbose = true

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population

elif make_error_bars:
    print("making error-bar file...")
    print(make_error_bar_file(file_in, file_out, qs))

elif make_table_of_D_numbers:
    print("making table of D numbers...")
    output_table_of_D_numbers(precomputed_error_bar_file, file_out, list_of_filenames, qs)

elif make_power_table:
    print("making power table...")
    error_envelope_x_vals, error_envelope_y_vals = read_error_bar_file(file_in, q)
    minimum_cells = get_minimum_cells(list_of_number_of_clones, list_of_fold_differences, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, min_number_of_doublets, fraction_small_clones)
    output_table(list_of_number_of_clones, list_of_fold_differences, file_out, number_of_error_bars, min_number_of_doublets, fraction_small_clones, minimum_cells, q, file_out)

elif resample:
    print("resampling...")
    output_resample(file_in, file_out)

elif make_resample_plot:
    print("plotting resample...")
    plot_resample(file_in, file_out, precomputed_error_bar_file)

elif upper_bound:
	print("calculating strict upper limit U...")
	print(get_upper_bound(fit_file, D_number_file, N, sample_size))


else:
    print("""error: no option given. Options:
-R  --run_recon
-e  --make_error_bars
-p  --make_power_table
-T  --make_table_of_D_numbers
-r  --resample
-x  --make_resample_plot
exiting...
""")
    exit()

running recon on df_fill_in_test.tsv...
==== BEGIN FIT ===
initial parameters [1.0, 0.06342727504755778]
iteration 1
old estimated n0	60154061
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.06342727504755778]
new estimated n_0	60154061
score_function 0.000000
Log likelihood	-3910522.491696

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.06342727504755778]
Final estimated_n0	60154061
Log likelihood	-3910522.491696
Observed clones	3941522.000000
Observed clones + estimated clones	64095583.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.28542273771401]
iteration 1
old estimated n0	11924670
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.28542273771401]
new estimated n_0	11924670
score_function 0.000000
Log likelihood	-2950742.221565

Completed iteration of clone sizes
Fitted weights	[1.0]
Fit

/usr/local/lib/python3.9/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Streaming output truncated to the last 5000 lines.
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.05, 0.0475, 0.045125, 0.04286875000000001, 0.45670528959678125, 0.21468057624193124, 0.08587223049677249, 0.0028624076832257507, 0.05438574598128927] [1.3954000510462712, 0.005946307035708542, 0.011892614071417083, 0.023785228142834167, 0.04757045628566833, 0.06342727504755778, 0.142711368857005, 1.3954000510462712, 0.5074182003804623]
new estimated n_0	32177593
score_function 0.000000
Log likelihood	-2548393.838232

Completed iteration of clone sizes
Fitted weights	[0.05, 0.0475, 0.045125, 0.04286875000000001, 0.45670528959678125, 0.21468057624193124, 0.08587223049677249, 0.0028624076832257507, 0.05438574598128927]
Fitted means	[1.3954000510462712, 0.005946307035708542, 0.01189

##### 1.4 ALL verbose = true

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population

elif make_error_bars:
    print("making error-bar file...")
    print(make_error_bar_file(file_in, file_out, qs))

elif make_table_of_D_numbers:
    print("making table of D numbers...")
    output_table_of_D_numbers(precomputed_error_bar_file, file_out, list_of_filenames, qs)

elif make_power_table:
    print("making power table...")
    error_envelope_x_vals, error_envelope_y_vals = read_error_bar_file(file_in, q)
    minimum_cells = get_minimum_cells(list_of_number_of_clones, list_of_fold_differences, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, min_number_of_doublets, fraction_small_clones)
    output_table(list_of_number_of_clones, list_of_fold_differences, file_out, number_of_error_bars, min_number_of_doublets, fraction_small_clones, minimum_cells, q, file_out)

elif resample:
    print("resampling...")
    output_resample(file_in, file_out)

elif make_resample_plot:
    print("plotting resample...")
    plot_resample(file_in, file_out, precomputed_error_bar_file)

elif upper_bound:
	print("calculating strict upper limit U...")
	print(get_upper_bound(fit_file, D_number_file, N, sample_size))


else:
    print("""error: no option given. Options:
-R  --run_recon
-e  --make_error_bars
-p  --make_power_table
-T  --make_table_of_D_numbers
-r  --resample
-x  --make_resample_plot
exiting...
""")
    exit()

running recon on df_fill_in_1all.tsv...
==== BEGIN FIT ===
initial parameters [1.0, 0.09587260570870364]
iteration 1
old estimated n0	515966480
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.09587260570870364]
new estimated n_0	515966480
score_function 0.000000
Log likelihood	-107640219.006563

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.09587260570870364]
Final estimated_n0	515966480
Log likelihood	-107640219.006563
Observed clones	52152541.000000
Observed clones + estimated clones	568119021.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.43142672568916635]
iteration 1
old estimated n0	96238253
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.43142672568916635]
new estimated n_0	96238253
score_function 0.000000
Log likelihood	-74883331.044286

Completed iteration of clone sizes
Fitted w

/usr/local/lib/python3.9/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Streaming output truncated to the last 5000 lines.
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.05000000000000002, 0.1221428585, 0.4967142849000001, 0.016557142830000003, 0.18875142826200003, 0.07550057130480001, 0.0025166857101600014, 0.028690217095824005, 0.019126811397216007] [2.10919732559148, 0.04793630285435182, 0.09587260570870364, 2.10919732559148, 0.09587260570870364, 0.23968151427175907, 2.10919732559148, 0.21571336284458317, 1.1025349656500916]
new estimated n_0	265595375
score_function 0.000000
Log likelihood	-54825689.635641

Completed iteration of clone sizes
Fitted weights	[0.05000000000000002, 0.1221428585, 0.4967142849000001, 0.016557142830000003, 0.18875142826200003, 0.07550057130480001, 0.0025166857101600014, 0.028690217095824005, 0.019126811397216007]
F

In [ ]:
52152541.000000/456588311.000000
# estimation is that we're only seeing ~11% of the total sample

0.11422224297809498

In [ ]:
-2346821.981794 # negative log-likelihood from df1-5
-52238577.832852 # negative log-likelihood from df_all
round(-52238577.832852 + 2346821.981794,0) # 49million different

-49891756.0

This negative log likelihood is lower so that means it's a better prediction. That makes sense because this is from all 20 dataframes whereas the first is only from 5 dataframes.

In [ ]:
58262035.000000 # estimated total clones from df1-5
456588311.000000 # estimated total clones from df_all
456588311 + 58262035 # 514,000,000 different (~0.5 million difference)

514850346

### 2 - Dataset 2

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
file_in = ['df_fill_in_2all.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 2.1 verbose = true

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population

elif make_error_bars:
    print("making error-bar file...")
    print(make_error_bar_file(file_in, file_out, qs))

elif make_table_of_D_numbers:
    print("making table of D numbers...")
    output_table_of_D_numbers(precomputed_error_bar_file, file_out, list_of_filenames, qs)

elif make_power_table:
    print("making power table...")
    error_envelope_x_vals, error_envelope_y_vals = read_error_bar_file(file_in, q)
    minimum_cells = get_minimum_cells(list_of_number_of_clones, list_of_fold_differences, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, min_number_of_doublets, fraction_small_clones)
    output_table(list_of_number_of_clones, list_of_fold_differences, file_out, number_of_error_bars, min_number_of_doublets, fraction_small_clones, minimum_cells, q, file_out)

elif resample:
    print("resampling...")
    output_resample(file_in, file_out)

elif make_resample_plot:
    print("plotting resample...")
    plot_resample(file_in, file_out, precomputed_error_bar_file)

elif upper_bound:
	print("calculating strict upper limit U...")
	print(get_upper_bound(fit_file, D_number_file, N, sample_size))


else:
    print("""error: no option given. Options:
-R  --run_recon
-e  --make_error_bars
-p  --make_power_table
-T  --make_table_of_D_numbers
-r  --resample
-x  --make_resample_plot
exiting...
""")
    exit()

running recon on df_fill_in_2all.tsv...
==== BEGIN FIT ===
initial parameters [1.0, 0.05568764273622112]
iteration 1
old estimated n0	14894062
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.05568764273622112]
new estimated n_0	14894062
score_function 0.000000
Log likelihood	-405826.590062

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.05568764273622112]
Final estimated_n0	14894062
Log likelihood	-405826.590062
Observed clones	852972.000000
Observed clones + estimated clones	15747034.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.2505943923129951]
iteration 1
old estimated n0	2995005
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.2505943923129951]
new estimated n_0	2995005
score_function 0.000000
Log likelihood	-346599.626876

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitte

/usr/local/lib/python3.9/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Streaming output truncated to the last 5000 lines.
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.12857143, 0.0435714285, 0.10643877651836735, 0.31948527385924197, 0.24115985467343437, 0.09646394186937375, 0.0032154647289791265, 0.06109382985060339] [1.0302213906200908, 0.00696095534202764, 0.01392191068405528, 0.02784382136811056, 0.05568764273622112, 0.05568764273622112, 1.0302213906200908, 0.2505943923129951]
new estimated n_0	5856547
score_function 0.000000
Log likelihood	-437791.484763

Completed iteration of clone sizes
Fitted weights	[0.12857143, 0.0435714285, 0.10643877651836735, 0.31948527385924197, 0.24115985467343437, 0.09646394186937375, 0.00321

### 3 - TCRdb:

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_tcr.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 3.1 verbose = True

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population

elif make_error_bars:
    print("making error-bar file...")
    print(make_error_bar_file(file_in, file_out, qs))

elif make_table_of_D_numbers:
    print("making table of D numbers...")
    output_table_of_D_numbers(precomputed_error_bar_file, file_out, list_of_filenames, qs)

elif make_power_table:
    print("making power table...")
    error_envelope_x_vals, error_envelope_y_vals = read_error_bar_file(file_in, q)
    minimum_cells = get_minimum_cells(list_of_number_of_clones, list_of_fold_differences, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, min_number_of_doublets, fraction_small_clones)
    output_table(list_of_number_of_clones, list_of_fold_differences, file_out, number_of_error_bars, min_number_of_doublets, fraction_small_clones, minimum_cells, q, file_out)

elif resample:
    print("resampling...")
    output_resample(file_in, file_out)

elif make_resample_plot:
    print("plotting resample...")
    plot_resample(file_in, file_out, precomputed_error_bar_file)

elif upper_bound:
	print("calculating strict upper limit U...")
	print(get_upper_bound(fit_file, D_number_file, N, sample_size))


else:
    print("""error: no option given. Options:
-R  --run_recon
-e  --make_error_bars
-p  --make_power_table
-T  --make_table_of_D_numbers
-r  --resample
-x  --make_resample_plot
exiting...
""")
    exit()

Streaming output truncated to the last 5000 lines.
iteration 1
old estimated n0	0
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.44285714, 0.55714286] [141.90716747215802, 19.176644252994326]
new estimated n_0	0
score_function 0.000000
Log likelihood	-202563.611050

Completed iteration of clone sizes
Fitted weights	[0.44285714, 0.55714286]
Fitted means	[141.90716747215802, 19.176644252994326]
Final estimated_n0	0
Log likelihood	-202563.611050
Observed clones	142770.000000
Observed clones + estimated clones	142770.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [0.52142857, 0.47857143, 141.90716747215802, 19.176644252994326]
iteration 1
old estimated n0	0
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.52142857, 0.47857143] [141.90716747215

## Uniform Distribution 1

### Uniform 10,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif_10_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 4.1 verbose = True

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population

elif make_error_bars:
    print("making error-bar file...")
    print(make_error_bar_file(file_in, file_out, qs))

elif make_table_of_D_numbers:
    print("making table of D numbers...")
    output_table_of_D_numbers(precomputed_error_bar_file, file_out, list_of_filenames, qs)

elif make_power_table:
    print("making power table...")
    error_envelope_x_vals, error_envelope_y_vals = read_error_bar_file(file_in, q)
    minimum_cells = get_minimum_cells(list_of_number_of_clones, list_of_fold_differences, error_envelope_x_vals, error_envelope_y_vals, number_of_error_bars, min_number_of_doublets, fraction_small_clones)
    output_table(list_of_number_of_clones, list_of_fold_differences, file_out, number_of_error_bars, min_number_of_doublets, fraction_small_clones, minimum_cells, q, file_out)

elif resample:
    print("resampling...")
    output_resample(file_in, file_out)

elif make_resample_plot:
    print("plotting resample...")
    plot_resample(file_in, file_out, precomputed_error_bar_file)

elif upper_bound:
	print("calculating strict upper limit U...")
	print(get_upper_bound(fit_file, D_number_file, N, sample_size))


else:
    print("""error: no option given. Options:
-R  --run_recon
-e  --make_error_bars
-p  --make_power_table
-T  --make_table_of_D_numbers
-r  --resample
-x  --make_resample_plot
exiting...
""")
    exit()

running recon on df_fill_in_unif_10_000.tsv...
1.5460234556502516e-09
14311 9382.0
==== BEGIN FIT ===
initial parameters [1.0, 0.05329354082285227]
iteration 1
old estimated n0	171395
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.05329354082285227]
new estimated n_0	171395
score_function 0.000000
Log likelihood	-2503.342607

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.05329354082285227]
Final estimated_n0	171395
Log likelihood	-2503.342607
Observed clones	9382.000000
Observed clones + estimated clones	180777.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.23982093370283522]
iteration 1
old estimated n0	34617
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.23982093370283522]
new estimated n_0	34617
score_function 0.000000
Log likelihood	-2470.184967

Completed iteration of clone sizes
F

### Uniform 100,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif_100_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 5.1 Verbose = True

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_unif_100_000.tsv...
2.9124949091787086e-07
69413 57295.0
==== BEGIN FIT ===
initial parameters [1.0, 0.08726764988218867]
iteration 1
old estimated n0	628313
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.08726764988218867]
new estimated n_0	628313
score_function 0.000000
Log likelihood	-144360.442718

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.08726764988218867]
Final estimated_n0	628313
Log likelihood	-144360.442718
Observed clones	57295.000000
Observed clones + estimated clones	685608.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.39270442446984904]
iteration 1
old estimated n0	119121
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.39270442446984904]
new estimated n_0	119121
score_function 0.000000
Log likelihood	-89228.326297

Completed iteration of clo

### Uniform 1,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif_1_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 6.1 Verbose = True

In [ ]:
test1 = pd.read_csv('df_fill_in_unif_1_000_000.tsv', sep='\t', header=None)

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_unif_1_000_000.tsv...
96327 96324.0
==== BEGIN FIT ===
initial parameters [1.0, 0.5190814334952868]
iteration 1
old estimated n0	141463
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.5190814334952868]
new estimated n_0	141463
score_function 0.000000
Log likelihood	-2332160.416817

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.5190814334952868]
Final estimated_n0	141463
Log likelihood	-2332160.416817
Observed clones	96324.000000
Observed clones + estimated clones	237787.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 2.3358664507287905]
iteration 1
old estimated n0	10308
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [2.3358664507287905]
new estimated n_0	10308
score_function 0.000000
Log likelihood	-1083030.487869

Completed iteration of clone sizes
Fitted weights	

### Uniform 3,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif_3_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 7.1 Verbose = True

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


Streaming output truncated to the last 5000 lines.
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.05000000000000002, 0.047500000000000014, 0.116035715575, 0.101116837692449, 0.03426737233662756, 0.08371029620959028, 0.16210565334196567, 0.13171084057441942, 0.27355328426994807] [33.15250150693189, 1.5069318866787222, 12.055455093429778, 22.60397830018083, 1.5069318866787222, 7.299201326100061, 22.60397830018083, 5.085895117540687, 17.329716696805303]
new estimated n_0	951
score_function 0.000000
Log likelihood	-170499.627486

Completed iteration of clone sizes
Fitted weights	[0.05000000000000002, 0.047500000000000014, 0.116035715575, 0.101116837692449, 0.03426737233662756, 0.08371029620959028, 0.16210565334196567, 0.13171084057441942, 0.27355328426994807]
Fitted means	[33.15

## Mixed Poisson Distribution 1

### Mixed Poisson 10,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_mp_10000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 8.1 Verbose = True

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_mp_10000.tsv...
14748 9564.0
==== BEGIN FIT ===
initial parameters [1.0, 0.05227938101212881]
iteration 1
old estimated n0	178200
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.05227938101212881]
new estimated n_0	178200
score_function 0.000000
Log likelihood	-1845.275154

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.05227938101212881]
Final estimated_n0	178200
Log likelihood	-1845.275154
Observed clones	9564.000000
Observed clones + estimated clones	187764.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.23525721455457965]
iteration 1
old estimated n0	36059
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.23525721455457965]
new estimated n_0	36059
score_function 0.000000
Log likelihood	-2085.453481

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0



Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.12857143, 0.87142857] [0.05227938101212881, 0.05227938101212881]
new estimated n_0	178200
score_function 0.000000
Log likelihood	-1845.275154

Completed iteration of clone sizes
Fitted weights	[0.12857143, 0.87142857]
Fitted means	[0.05227938101212881, 0.05227938101212881]
Final estimated_n0	178200
Log likelihood	-1845.275154
Observed clones	9564.000000
Observed clones + estimated clones	187764.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [0.20714286, 0.79285714, 0.05227938101212881, 0.05227938101212881]
iteration 1
old estimated n0	178200
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.20714286, 0.79285714] [0.05227938101212881, 0.05227938101212881]
new estimated n_0	178200
score_function 0.000000
Log likelihood	-1845.275154

Completed iteration of clone sizes
Fitted weights	[0.2071

### Mixed Poisson 100,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_mp_100000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 9.1 Verbose = True

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_mp_100000.tsv...
84498 65949.0
==== BEGIN FIT ===
initial parameters [1.0, 0.07581616097287298]
iteration 1
old estimated n0	837296
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.07581616097287298]
new estimated n_0	837296
score_function 0.000000
Log likelihood	-117536.141055

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.07581616097287298]
Final estimated_n0	837296
Log likelihood	-117536.141055
Observed clones	65949.000000
Observed clones + estimated clones	903245.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.34117272437792845]
iteration 1
old estimated n0	162198
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.34117272437792845]
new estimated n_0	162198
score_function 0.000000
Log likelihood	-75374.547091

Completed iteration of clone sizes
Fitted weights	[1

### Mixed Poisson 300,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_mp_300000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))


In [ ]:
# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()


#### 10.1 Verbose = True

In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_mp_300000.tsv...
103164 98289.0
==== BEGIN FIT ===
initial parameters [1.0, 0.15261117724262127]
iteration 1
old estimated n0	596154
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.15261117724262127]
new estimated n_0	596154
score_function 0.000000
Log likelihood	-592619.785913

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.15261117724262127]
Final estimated_n0	596154
Log likelihood	-592619.785913
Observed clones	98289.000000
Observed clones + estimated clones	694443.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.6867502975917956]
iteration 1
old estimated n0	99559
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.6867502975917956]
new estimated n_0	99559
score_function 0.000000
Log likelihood	-317309.417039

Completed iteration of clone sizes
Fitted weights	[1.0

## Uniform Distribution 2

### Uniform 10,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif1_10_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_unif1_10_000.tsv...
15638 9927.0
==== BEGIN FIT ===
initial parameters [1.0, 0.05036768409388537]
iteration 1
old estimated n0	192169
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.05036768409388537]
new estimated n_0	192169
score_function 0.000000
Log likelihood	-519.802650

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.05036768409388537]
Final estimated_n0	192169
Log likelihood	-519.802650
Observed clones	9927.000000
Observed clones + estimated clones	202096.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.22665457842248415]
iteration 1
old estimated n0	39022
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.22665457842248415]
new estimated n_0	39022
score_function 0.000000
Log likelihood	-1305.195473

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitt

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Completed iteration of clone sizes
Fitted weights	[0.20714286, 0.79285714]
Fitted means	[0.05036768409388537, 0.025183842046942684]
Final estimated_n0	322160
Log likelihood	-466.021338
Observed clones	9927.000000
Observed clones + estimated clones	332087.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [0.28571429, 0.71428571, 0.05036768409388537, 0.025183842046942684]
iteration 1
old estimated n0	302260
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.28571429, 0.71428571] [0.05036768409388537, 0.025183842046942684]
new estimated n_0	302260
score_function 0.000000
Log likelihood	-473.406570

Completed iteration of clone sizes
Fitted weights	[0.28571429, 0.71428571]
Fitted means	[0.05036768409388537, 0.025183842046942684]
Final estimated_n0	302260
Log likelihood	-473.406570
Observed clones	9927.000000
Observed clones + estimated clones	312187.000000
=== END 

### Uniform 100,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif1_100_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_unif1_100_000.tsv...
142981 93765.0
==== BEGIN FIT ===
initial parameters [1.0, 0.053324801365114916]
iteration 1
old estimated n0	1711909
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.053324801365114916]
new estimated n_0	1711909
score_function 0.000000
Log likelihood	-25238.953009

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.053324801365114916]
Final estimated_n0	1711909
Log likelihood	-25238.953009
Observed clones	93765.000000
Observed clones + estimated clones	1805674.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.23996160614301712]
iteration 1
old estimated n0	345741
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.23996160614301712]
new estimated n_0	345741
score_function 0.000000
Log likelihood	-24824.777526

Completed iteration of clone sizes
Fitted 

### Uniform 1,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif1_1_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


Streaming output truncated to the last 5000 lines.
iteration 1
old estimated n0	180031
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.44285714, 0.55714286] [1.6129454126960603, 1.3077935778616703]
new estimated n_0	180031
score_function 0.000000
Log likelihood	-683295.064536

Completed iteration of clone sizes
Fitted weights	[0.44285714, 0.55714286]
Fitted means	[1.6129454126960603, 1.3077935778616703]
Final estimated_n0	180031
Log likelihood	-683295.064536
Observed clones	573485.000000
Observed clones + estimated clones	753516.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [0.52142857, 0.47857143, 1.6129454126960603, 1.3077935778616703]
iteration 1
old estimated n0	174540
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.52142857, 0.478571

### Uniform 3,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif1_3_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_unif1_3_000_000.tsv...
870035 845046.0
==== BEGIN FIT ===
initial parameters [1.0, 0.17750512989825407]
iteration 1
old estimated n0	4350655
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.17750512989825407]
new estimated n_0	4350655
score_function 0.000000
Log likelihood	-6409468.597710

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.17750512989825407]
Final estimated_n0	4350655
Log likelihood	-6409468.597710
Observed clones	845046.000000
Observed clones + estimated clones	5195701.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.7987730845421434]
iteration 1
old estimated n0	691068
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.7987730845421434]
new estimated n_0	691068
score_function 0.000000
Log likelihood	-3451989.119012

Completed iteration of clone sizes
Fi

### Uniform 10,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif1_10_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_unif1_10_000_000.tsv...
962797 962767.0
==== BEGIN FIT ===
initial parameters [1.0, 0.5193364541991988]
iteration 1
old estimated n0	1413052
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.5193364541991988]
new estimated n_0	1413052
score_function 0.000000
Log likelihood	-23313768.609945

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.5193364541991988]
Final estimated_n0	1413052
Log likelihood	-23313768.609945
Observed clones	962767.000000
Observed clones + estimated clones	2375819.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 2.337014043896395]
iteration 1
old estimated n0	102902
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [2.337014043896395]
new estimated n_0	102902
score_function 0.000000
Log likelihood	-10821564.504670

Completed iteration of clone sizes
Fit

### Uniform 20,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif1_20_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


Streaming output truncated to the last 5000 lines.
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.05, 0.04750000000000001, 0.045125000000000005, 0.11023392979624999, 0.0373570535101875, 0.09125794601743521, 0.22532020880976317, 0.11234453364698628, 0.2808613282193778] [22.276901890498902, 8.100691596545055, 4.3034924106645605, 22.276901890498902, 1.0125864495681318, 8.100691596545055, 22.276901890498902, 4.556639023056594, 15.188796743521976]
new estimated n_0	11523
score_function 0.000000
Log likelihood	-2531490.622773

Completed iteration of clone sizes
Fitted weights	[0.05, 0.04750000000000001, 0.045125000000000005, 0.11023392979624999, 0.0373570535101875, 0.09125794601743521, 0.22532020880976317, 0.11234453364698628, 0.2808613282193778]
Fitted means	[22.276901890498902, 

### Uniform 30,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif1_30_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


Streaming output truncated to the last 5000 lines.
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.05, 0.0475, 0.11603571557499998, 0.07022002595684948, 0.03581221292340753, 0.08748412111351271, 0.1694136952358426, 0.13764862448850096, 0.2858856047068867] [33.166795816159464, 1.5075816280072485, 22.613724420108724, 11.495309913555268, 0.7537908140036242, 6.784117326032618, 22.613724420108724, 5.0880879945244635, 17.337188722083354]
new estimated n_0	14275
score_function 0.000000
Log likelihood	-1701217.434612

Completed iteration of clone sizes
Fitted weights	[0.05, 0.0475, 0.11603571557499998, 0.07022002595684948, 0.03581221292340753, 0.08748412111351271, 0.1694136952358426, 0.13764862448850096, 0.2858856047068867]
Fitted means	[33.166795816159464, 1.5075816280072485, 22.613

### Uniform 40,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_unif1_40_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


Streaming output truncated to the last 5000 lines.
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [0.05, 0.047500000000000014, 0.04512500000000001, 0.11023392979625, 0.15476493810546554, 0.145978406096992, 0.1275422093420738, 0.0660486436475682, 0.25280687301165045] [44.072278536800354, 23.037781962418364, 23.037781962418364, 9.014784246163709, 23.037781962418364, 5.008213470090949, 23.037781962418364, 2.0032853880363795, 15.525461757281942]
new estimated n_0	3657
score_function 0.000000
Log likelihood	-1272736.167886

Completed iteration of clone sizes
Fitted weights	[0.05, 0.047500000000000014, 0.04512500000000001, 0.11023392979625, 0.15476493810546554, 0.145978406096992, 0.1275422093420738, 0.0660486436475682, 0.25280687301165045]
Fitted means	[44.072278536800354, 23.0377819

## Mixed Poisson Distribution 2

### Mixed Poisson 10,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_mp1_10_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_mp1_10_000.tsv...
15697 9951.0
==== BEGIN FIT ===
initial parameters [1.0, 0.050246206411415945]
iteration 1
old estimated n0	193111
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.050246206411415945]
new estimated n_0	193111
score_function 0.000000
Log likelihood	-432.372107

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.050246206411415945]
Final estimated_n0	193111
Log likelihood	-432.372107
Observed clones	9951.000000
Observed clones + estimated clones	203062.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.22610792885137174]
iteration 1
old estimated n0	39222
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.22610792885137174]
new estimated n_0	39222
score_function 0.000000
Log likelihood	-1253.814135

Completed iteration of clone sizes
Fitted weights	[1.0]
Fit

### Mixed Poisson 100,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_mp1_100_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_mp1_100_000.tsv...
147756 95756.0
==== BEGIN FIT ===
initial parameters [1.0, 0.052216049124859026]
iteration 1
old estimated n0	1786381
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.052216049124859026]
new estimated n_0	1786381
score_function 0.000000
Log likelihood	-18024.483978

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.052216049124859026]
Final estimated_n0	1786381
Log likelihood	-18024.483978
Observed clones	95756.000000
Observed clones + estimated clones	1882137.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.2349722210618656]
iteration 1
old estimated n0	361516
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.2349722210618656]
new estimated n_0	361516
score_function 0.000000
Log likelihood	-20600.486602

Completed iteration of clone sizes
Fitted weig

### Mixed Poisson 300,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_mp1_300_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_mp1_300_000.tsv...
388047 263668.0
==== BEGIN FIT ===
initial parameters [1.0, 0.056889724957143076]
iteration 1
old estimated n0	4504137
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.056889724957143076]
new estimated n_0	4504137
score_function 0.000000
Log likelihood	-138007.900583

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.056889724957143076]
Final estimated_n0	4504137
Log likelihood	-138007.900583
Observed clones	263668.000000
Observed clones + estimated clones	4767805.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.25600376230714383]
iteration 1
old estimated n0	903723
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.25600376230714383]
new estimated n_0	903723
score_function 0.000000
Log likelihood	-110295.823241

Completed iteration of clone sizes
Fitt

### Mixed Poisson 1,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_mp1_1_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_mp1_1_000_000.tsv...
844932 659465.0
==== BEGIN FIT ===
initial parameters [1.0, 0.07581903512695898]
iteration 1
old estimated n0	8372315
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.07581903512695898]
new estimated n_0	8372315
score_function 0.000000
Log likelihood	-1175428.898132

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.07581903512695898]
Final estimated_n0	8372315
Log likelihood	-1175428.898132
Observed clones	659465.000000
Observed clones + estimated clones	9031780.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.3411856580713154]
iteration 1
old estimated n0	1621844
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.3411856580713154]
new estimated n_0	1621844
score_function 0.000000
Log likelihood	-753775.471467

Completed iteration of clone sizes
Fit

### Mixed Poisson 2,000,000

In [ ]:
# THIS CODE HAS TO BE RUN FIRST IN THE NOTEBOOK (AFTER THE IMPORTS)
# THEN THE CLASSES/FUNCTIONS CAN BE RUN - FUNCTIONS DEPEND ON THESE VARIABLES
# THIS IS ADAPTED FROM THE PARSER SECTION

aicc_multiple = 1.3
precomputed_error_bar_file = '' #file name for precomputed error bars
###
clone_distribution_in_file = True # input file is of the form clone_size tab number_of_clones of that size
###
bin_size = 1 # average number of observations for each individual
D_number_file = None # None or string, output of -D option
make_error_bars = False # make a precomputed error bar file that can be used to calculate error bars
fraction_small_clones = 0.1 # see get_sampling_limit
fit_file = None # Recon fit-file (output of -R option)
javascript_file = 'plot_clone_size_distribution_ref.js' # reference js file, for plotting resample
parameter_limit = 20 # maximum number of parameters to fit
min_number_of_doublets = 100 # see get_sampling_limit
number_of_error_bars = 2 # mapping between gold-standard error bar fits and standard deviations in power calculation
file_out = 'test_recon_output.txt' # output file for fit
make_power_table = False # ouput a table of cells required to see fold difference
q = 0. # Hill number parameter to use when making table for power calculations
resample = False # output a resampling of the model fit with the given fit file
noise_test_ratio_threshold = 3. # for avoiding fitting noise
sample_size = None # sample size, for calculating U_max
threshold = 30 # largest clone to fit using parameters. Any larger clones will be assumed to be statistically accurate between sample and overall population and added back in after the fit is performed
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
cutoff_score = 0.1 # for stopping iteration when estimated clone distribution is at a fixed point
###
verbose = True # print verbose output to stdout
very_verbose = False # print even more verbose output to stdout
###
make_resample_plot = False # plots resample of fit against original sample
total_clones_in_overall_population = None # total clones in an overall distribution. for use with plot_resample (optionally) and reconstruct_overall_distribution(optionally)
list_of_number_of_clones = [1e4, 3e4, 1e5, 1e6, 3e6] # column headers for power table
make_table_of_D_numbers = False # output a table of reconstructed D numbers with error bars from input fit files
list_of_fold_differences = [1.1, 1.2, 1.3, 1.4, 1.5, 2., 5.] # row titles for power table
N = None # overall population size, for calculating U_max
qs = [0., 1., 2., float('inf')] # space-separated list of Hill-number parameters for making error-bar files and table of D numbers
run_recon = True # run Recon on sample to generate overall distribution parameters and fit file
initial_scale_factors = [0.05, 0.225, 0.4, 0.575, 0.75 , 0.925, 1.1] # for determining means
resource_path = PATH # folder that contains style.css, d3.js, and plot_clone_size_distribution_ref.js
upper_bound = False # calculate U_max
test_weights_list = [0.05, 0.12857143, 0.20714286, 0.28571429, 0.36428571, 0.44285714, 0.52142857, 0.6] # list of weights to attempt to add
x_max = None # the x_max for the output plot
y_max = None # the y_max for the output plot (note, log scale)
###
file_in = ['df_fill_in_mp1_2_000_000.tsv'] # list of file names containing clone data. a long list of files can be conveniently input using a shell variable, e.g. MLE_files=$(ls);python recon.py file_out $MLE_files
###

if very_verbose: verbose = True

list_of_number_of_clones = list(map(float, list_of_number_of_clones))
list_of_fold_differences = list(map(float, list_of_fold_differences))
initial_scale_factors = list(map(float, initial_scale_factors))
test_weights_list = list(map(float, test_weights_list))
qs = list(map(float, qs))

## Formatting

# Handle file/directory names

if make_table_of_D_numbers:
    list_of_filenames = file_in

elif len(file_in) == 1:
    file_in = file_in[0]
    if make_error_bars:
        if not isdir(file_in):
            print('error: input file %s is not a directory. Input must be a directory containing the error bar fits when making error-bar files. exiting...' % file_in)
            exit()
        else:
            error_bar_fits_directory = file_in
            if not error_bar_fits_directory.endswith('/'):
                error_bar_fits_directory = error_bar_fits_directory + '/'
    else:
        if not isfile(file_in):
            print('error: input file %s does not exist. exiting...' % file_in)
            exit()

elif len(file_in) > 1:
    print('error: multiple files input, but this is allowed only for outputting table of D numbers. check the order of command-line parameters. exiting...')
    exit()



In [ ]:
if run_recon: 
    print("running recon on %s..." % file_in)
    print(reconstruct_overall_distribution(file_in, file_out, total_clones_in_overall_population)) 
    # total_clones_in_overall_population passed just to record in
    # file_out. When this is a fit of a known overall distribution,
    # e.g. for error-bar fits, we will need this record of how many
    # clones there were in the overall population


running recon on df_fill_in_mp1_2_000_000.tsv...
1018711 906533.0
==== BEGIN FIT ===
initial parameters [1.0, 0.11031038031709822]
iteration 1
old estimated n0	7773086
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.11031038031709822]
new estimated n_0	7773086
score_function 0.000000
Log likelihood	-3454156.384256

Completed iteration of clone sizes
Fitted weights	[1.0]
Fitted means	[0.11031038031709822]
Final estimated_n0	7773086
Log likelihood	-3454156.384256
Observed clones	906533.000000
Observed clones + estimated clones	8679619.000000
=== END FIT ===

==== BEGIN FIT ===
initial parameters [1.0, 0.496396711426942]
iteration 1
old estimated n0	1410307
Log likelihood is -inf!
Log likelihood is -inf!
Log likelihood is -inf!
Fitted parameters (new_weights, new_means) [1.0] [0.496396711426942]
new estimated n_0	1410307
score_function 0.000000
Log likelihood	-1993326.206172

Completed iteration of clone sizes
Fit